## Load in packages

In [ ]:
library(data.table)
library(ggplot2)
library(corrplot)
library(patchwork)
library(gtools)
library(tableone)
library(RColorBrewer)
library(survival)
library(survminer)
library(pROC)
library(tableone)
library(splines)
library(forestplot)

## Write custom functions

In [ ]:
saaketSumm <- function(dat){
    print(dim(dat))
    print(names(dat))
    return(head(dat))
}

unlogit <- function(val){
    return((exp(val)/(1+exp(val))))
}

## Read in data and define new variables

In [ ]:
fatPhenoCovarLifestyle2 <- fread('/medpop/esp2/sagrawal/fat_distribution/2021.03.11_datafreeze/fatPhenoCovarWithLifestyle2.csv')

In [ ]:
names(fatPhenoCovarLifestyle2)

In [ ]:
fatPhenoCovarLifestyle2$ethnicity_group <- factor(fatPhenoCovarLifestyle2$ethnicity_group,
                                                 levels=c('White','Black','East Asian','South Asian','Other'))

In [ ]:
#Define BMI adjusted parameters

fatPhenoCovarLifestyle2[sex=='Male',vatadjbmi:=residuals(lm(vat~bmiCalc,fatPhenoCovarLifestyle2[sex=='Male'],na.action = na.exclude))]
fatPhenoCovarLifestyle2[sex=='Female',vatadjbmi:=residuals(lm(vat~bmiCalc,fatPhenoCovarLifestyle2[sex=='Female'],na.action = na.exclude))]

fatPhenoCovarLifestyle2[sex=='Male',asatadjbmi:=residuals(lm(asat~bmiCalc,fatPhenoCovarLifestyle2[sex=='Male'],na.action = na.exclude))]
fatPhenoCovarLifestyle2[sex=='Female',asatadjbmi:=residuals(lm(asat~bmiCalc,fatPhenoCovarLifestyle2[sex=='Female'],na.action = na.exclude))]

fatPhenoCovarLifestyle2[sex=='Male',gfatadjbmi:=residuals(lm(gfat~bmiCalc,fatPhenoCovarLifestyle2[sex=='Male'],na.action = na.exclude))]
fatPhenoCovarLifestyle2[sex=='Female',gfatadjbmi:=residuals(lm(gfat~bmiCalc,fatPhenoCovarLifestyle2[sex=='Female'],na.action = na.exclude))]

#As a sensitivity analysis, we additionally define BMI adjusted parameters using a spline basis

fatPhenoCovarLifestyle2[sex=='Male',vatadjbmiSpline:=residuals(lm(vat~bs(bmiCalc,knots=c(25,30,35)),fatPhenoCovarLifestyle2[sex=='Male'],na.action = na.exclude))]
fatPhenoCovarLifestyle2[sex=='Female',vatadjbmiSpline:=residuals(lm(vat~bs(bmiCalc,knots=c(25,30,35)),fatPhenoCovarLifestyle2[sex=='Female'],na.action = na.exclude))]

fatPhenoCovarLifestyle2[sex=='Male',asatadjbmiSpline:=residuals(lm(asat~bs(bmiCalc,knots=c(25,30,35)),fatPhenoCovarLifestyle2[sex=='Male'],na.action = na.exclude))]
fatPhenoCovarLifestyle2[sex=='Female',asatadjbmiSpline:=residuals(lm(asat~bs(bmiCalc,knots=c(25,30,35)),fatPhenoCovarLifestyle2[sex=='Female'],na.action = na.exclude))]

fatPhenoCovarLifestyle2[sex=='Male',gfatadjbmiSpline:=residuals(lm(gfat~bs(bmiCalc,knots=c(25,30,35)),fatPhenoCovarLifestyle2[sex=='Male'],na.action = na.exclude))]
fatPhenoCovarLifestyle2[sex=='Female',gfatadjbmiSpline:=residuals(lm(gfat~bs(bmiCalc,knots=c(25,30,35)),fatPhenoCovarLifestyle2[sex=='Female'],na.action = na.exclude))]

In [ ]:
fatNames <- names(fatPhenoCovarLifestyle2)[c(2:14,74,76:78)]

for(i in 1:length(fatNames)){
    fatPhenoCovarLifestyle2[,paste0(fatNames[i],'Scaled'):=scale(eval(as.name(fatNames[i]))),by=sex]
}

In [ ]:
fatPhenoCovarLifestyle2[bmiCalc < 25, bmiGroup:='Normal']
fatPhenoCovarLifestyle2[bmiCalc >= 25 & bmiCalc < 30, bmiGroup:='Overweight']
fatPhenoCovarLifestyle2[bmiCalc >= 30, bmiGroup:='Obese']

fatPhenoCovarLifestyle2[age_instance2 < 60, ageGroup:='Under 60']
fatPhenoCovarLifestyle2[age_instance2 >= 60 & age_instance2 < 70, ageGroup:='60 to 70']
fatPhenoCovarLifestyle2[age_instance2 >= 70, ageGroup:='Over 70']

fatPhenoCovarLifestyle2$bmiGroup <- factor(fatPhenoCovarLifestyle2$bmiGroup,levels=c('Normal','Overweight','Obese'))

fatPhenoCovarLifestyle2$ageGroup <- factor(fatPhenoCovarLifestyle2$ageGroup,levels=c('Under 60','60 to 70','Over 70'))

In [ ]:
# Sensitivity analysis requested in revision, add ethnicity specific cutoffs based on:
# https://diabetesjournals.org/care/article/37/9/2500/29021/Ethnic-Specific-Obesity-Cutoffs-for-Diabetes-Risk

fatPhenoCovarLifestyle2[bmiCalc < 30, bmiDichotJoint:='Low']
fatPhenoCovarLifestyle2[bmiCalc >= 30, bmiDichotJoint:='High']
fatPhenoCovarLifestyle2$bmiDichotJoint <- factor(fatPhenoCovarLifestyle2$bmiDichotJoint,
                                                levels=c('Low','High'))

fatPhenoCovarLifestyle2[ethnicity_group=='White'&bmiCalc < 30, bmiDichotEthnicity:='Low']
fatPhenoCovarLifestyle2[ethnicity_group=='White'&bmiCalc >= 30, bmiDichotEthnicity:='High']

fatPhenoCovarLifestyle2[ethnicity_group=='Black'&bmiCalc < 26, bmiDichotEthnicity:='Low']
fatPhenoCovarLifestyle2[ethnicity_group=='Black'&bmiCalc >= 26, bmiDichotEthnicity:='High']

fatPhenoCovarLifestyle2[ethnicity_group=='South Asian'&bmiCalc < 22, bmiDichotEthnicity:='Low']
fatPhenoCovarLifestyle2[ethnicity_group=='South Asian'&bmiCalc >= 22, bmiDichotEthnicity:='High']

fatPhenoCovarLifestyle2[ethnicity_group=='East Asian'&bmiCalc < 26, bmiDichotEthnicity:='Low']
fatPhenoCovarLifestyle2[ethnicity_group=='East Asian'&bmiCalc >= 24, bmiDichotEthnicity:='High']

fatPhenoCovarLifestyle2$bmiDichotEthnicity <- factor(fatPhenoCovarLifestyle2$bmiDichotEthnicity,
                                                levels=c('Low','High'))

In [ ]:
fatPhenoCovarLifestyle2[!is.na(bmiCalc), vatadjbmiQuintiles := quantcut(vatadjbmi,5,labels=paste0('Q',1:5)), by=c('sex','bmiGroup')]
fatPhenoCovarLifestyle2[!is.na(bmiCalc), asatadjbmiQuintiles := quantcut(asatadjbmi,5,labels=paste0('Q',1:5)), by=c('sex','bmiGroup')]
fatPhenoCovarLifestyle2[!is.na(bmiCalc), gfatadjbmiQuintiles := quantcut(gfatadjbmi,5,labels=paste0('Q',1:5)), by=c('sex','bmiGroup')]

fatPhenoCovarLifestyle2[vatadjbmiQuintiles=='Q1', vatadjbmiQuintilesStrat := 'Q1']
fatPhenoCovarLifestyle2[vatadjbmiQuintiles%in%c('Q2','Q3','Q4'), vatadjbmiQuintilesStrat := 'Q2_Q4']
fatPhenoCovarLifestyle2[vatadjbmiQuintiles=='Q5', vatadjbmiQuintilesStrat := 'Q5']
fatPhenoCovarLifestyle2$vatadjbmiQuintilesStrat <- factor(fatPhenoCovarLifestyle2$vatadjbmiQuintilesStrat)

fatPhenoCovarLifestyle2[asatadjbmiQuintiles=='Q1', asatadjbmiQuintilesStrat := 'Q1']
fatPhenoCovarLifestyle2[asatadjbmiQuintiles%in%c('Q2','Q3','Q4'), asatadjbmiQuintilesStrat := 'Q2_Q4']
fatPhenoCovarLifestyle2[asatadjbmiQuintiles=='Q5', asatadjbmiQuintilesStrat := 'Q5']
fatPhenoCovarLifestyle2$asatadjbmiQuintilesStrat <- factor(fatPhenoCovarLifestyle2$asatadjbmiQuintilesStrat)

fatPhenoCovarLifestyle2[gfatadjbmiQuintiles=='Q1', gfatadjbmiQuintilesStrat := 'Q1']
fatPhenoCovarLifestyle2[gfatadjbmiQuintiles%in%c('Q2','Q3','Q4'), gfatadjbmiQuintilesStrat := 'Q2_Q4']
fatPhenoCovarLifestyle2[gfatadjbmiQuintiles=='Q5', gfatadjbmiQuintilesStrat := 'Q5']
fatPhenoCovarLifestyle2$gfatadjbmiQuintilesStrat <- factor(fatPhenoCovarLifestyle2$gfatadjbmiQuintilesStrat)


## Compare BMI adjusted fat depots generated from (1) simple linear residualization vs. (2) spline-based residualization

In [ ]:
fatPhenoCovarLifestyle2[sex=='Male',vat_bmiPred:=predict(lm(vat~bmiCalc,fatPhenoCovarLifestyle2[sex=='Male'],na.action = na.exclude))]
fatPhenoCovarLifestyle2[sex=='Female',vat_bmiPred:=predict(lm(vat~bmiCalc,fatPhenoCovarLifestyle2[sex=='Female'],na.action = na.exclude))]

fatPhenoCovarLifestyle2[sex=='Male',asat_bmiPred:=predict(lm(asat~bmiCalc,fatPhenoCovarLifestyle2[sex=='Male'],na.action = na.exclude))]
fatPhenoCovarLifestyle2[sex=='Female',asat_bmiPred:=predict(lm(asat~bmiCalc,fatPhenoCovarLifestyle2[sex=='Female'],na.action = na.exclude))]

fatPhenoCovarLifestyle2[sex=='Male',gfat_bmiPred:=predict(lm(gfat~bmiCalc,fatPhenoCovarLifestyle2[sex=='Male'],na.action = na.exclude))]
fatPhenoCovarLifestyle2[sex=='Female',gfat_bmiPred:=predict(lm(gfat~bmiCalc,fatPhenoCovarLifestyle2[sex=='Female'],na.action = na.exclude))]


In [ ]:
fatPhenoCovarLifestyle2[sex=='Male',vat_bmiSplinePred:=predict(lm(vat~bs(bmiCalc,knots=c(25,30,35)),fatPhenoCovarLifestyle2[sex=='Male'],na.action = na.exclude))]
fatPhenoCovarLifestyle2[sex=='Female',vat_bmiSplinePred:=predict(lm(vat~bs(bmiCalc,knots=c(25,30,35)),fatPhenoCovarLifestyle2[sex=='Female'],na.action = na.exclude))]

fatPhenoCovarLifestyle2[sex=='Male',asat_bmiSplinePred:=predict(lm(asat~bs(bmiCalc,knots=c(25,30,35)),fatPhenoCovarLifestyle2[sex=='Male'],na.action = na.exclude))]
fatPhenoCovarLifestyle2[sex=='Female',asat_bmiSplinePred:=predict(lm(asat~bs(bmiCalc,knots=c(25,30,35)),fatPhenoCovarLifestyle2[sex=='Female'],na.action = na.exclude))]

fatPhenoCovarLifestyle2[sex=='Male',gfat_bmiSplinePred:=predict(lm(gfat~bs(bmiCalc,knots=c(25,30,35)),fatPhenoCovarLifestyle2[sex=='Male'],na.action = na.exclude))]
fatPhenoCovarLifestyle2[sex=='Female',gfat_bmiSplinePred:=predict(lm(gfat~bs(bmiCalc,knots=c(25,30,35)),fatPhenoCovarLifestyle2[sex=='Female'],na.action = na.exclude))]


In [ ]:
round(cor(fatPhenoCovarLifestyle2[sex=='Male'&!is.na(vatadjbmi),c('vatadjbmi','vatadjbmiSpline')])[1,2],3)
round(cor(fatPhenoCovarLifestyle2[sex=='Female'&!is.na(vatadjbmi),c('vatadjbmi','vatadjbmiSpline')])[1,2],3)

round(cor(fatPhenoCovarLifestyle2[sex=='Male'&!is.na(asatadjbmi),c('asatadjbmi','asatadjbmiSpline')])[1,2],3)
round(cor(fatPhenoCovarLifestyle2[sex=='Female'&!is.na(asatadjbmi),c('asatadjbmi','asatadjbmiSpline')])[1,2],3)

round(cor(fatPhenoCovarLifestyle2[sex=='Male'&!is.na(gfatadjbmi),c('gfatadjbmi','gfatadjbmiSpline')])[1,2],3)
round(cor(fatPhenoCovarLifestyle2[sex=='Female'&!is.na(gfatadjbmi),c('gfatadjbmi','gfatadjbmiSpline')])[1,2],3)

In [ ]:
options(repr.plot.width=10,repr.plot.height=15)

g1 <- ggplot(fatPhenoCovarLifestyle2[sex=='Male'],aes(bmiCalc,vat))+
geom_point()+
geom_line(aes(y=vat_bmiPred),color='red',size=1)+
geom_line(aes(y=vat_bmiSplinePred),color='blue',size=1)+
    xlab('BMI (kg/m2)') +
    ylab('VAT (Liters)') +
    theme_classic() +
#     coord_flip() +
#     scale_x_discrete(limits=rev(levels(riskBarPlotDat$bmiGroup))) +
    theme(axis.title.y = element_text(size=20),
          axis.text.y = element_text(size=18,color='black'),
          axis.ticks.y = element_blank(),
          axis.title.x = element_text(size=20),
          axis.text.x = element_text(size=18,color='black'),
          axis.ticks.x = element_blank(),
          legend.text.align=0,
         legend.key.size=unit(1,'cm'),
         legend.title = element_text(size=16),
         legend.text = element_text(size=16),
         legend.position='none')+
annotate("text",x=20,y=14.5,label=paste0('Male'),size=6)

g2 <- ggplot(fatPhenoCovarLifestyle2[sex=='Female'],aes(bmiCalc,vat))+
geom_point()+
geom_line(aes(y=vat_bmiPred),color='red',size=1)+
geom_line(aes(y=vat_bmiSplinePred),color='blue',size=1)+
    xlab('BMI (kg/m2)') +
    ylab('VAT (Liters)') +
    theme_classic() +
#     coord_flip() +
#     scale_x_discrete(limits=rev(levels(riskBarPlotDat$bmiGroup))) +
    theme(axis.title.y = element_text(size=20),
          axis.text.y = element_text(size=18,color='black'),
          axis.ticks.y = element_blank(),
          axis.title.x = element_text(size=20),
          axis.text.x = element_text(size=18,color='black'),
          axis.ticks.x = element_blank(),
          legend.text.align=0,
         legend.key.size=unit(1,'cm'),
         legend.title = element_text(size=16),
         legend.text = element_text(size=16),
         legend.position='none')+
annotate("text",x=20,y=9,label=paste0('Female'),size=6)

g3 <- ggplot(fatPhenoCovarLifestyle2[sex=='Male'],aes(bmiCalc,asat))+
geom_point()+
geom_line(aes(y=asat_bmiPred),color='red',size=1)+
geom_line(aes(y=asat_bmiSplinePred),color='blue',size=1)+
    xlab('BMI (kg/m2)') +
    ylab('ASAT (Liters)') +
    theme_classic() +
#     coord_flip() +
#     scale_x_discrete(limits=rev(levels(riskBarPlotDat$bmiGroup))) +
    theme(axis.title.y = element_text(size=20),
          axis.text.y = element_text(size=18,color='black'),
          axis.ticks.y = element_blank(),
          axis.title.x = element_text(size=20),
          axis.text.x = element_text(size=18,color='black'),
          axis.ticks.x = element_blank(),
          legend.text.align=0,
         legend.key.size=unit(1,'cm'),
         legend.title = element_text(size=16),
         legend.text = element_text(size=16),
         legend.position='none')+
annotate("text",x=20,y=20,label=paste0('Male'),size=6)

g4 <- ggplot(fatPhenoCovarLifestyle2[sex=='Female'],aes(bmiCalc,asat))+
geom_point()+
geom_line(aes(y=asat_bmiPred),color='red',size=1)+
geom_line(aes(y=asat_bmiSplinePred),color='blue',size=1)+
    xlab('BMI (kg/m2)') +
    ylab('ASAT (Liters)') +
    theme_classic() +
#     coord_flip() +
#     scale_x_discrete(limits=rev(levels(riskBarPlotDat$bmiGroup))) +
    theme(axis.title.y = element_text(size=20),
          axis.text.y = element_text(size=18,color='black'),
          axis.ticks.y = element_blank(),
          axis.title.x = element_text(size=20),
          axis.text.x = element_text(size=18,color='black'),
          axis.ticks.x = element_blank(),
          legend.text.align=0,
         legend.key.size=unit(1,'cm'),
         legend.title = element_text(size=16),
         legend.text = element_text(size=16),
         legend.position='none')+
annotate("text",x=20,y=23,label=paste0('Female'),size=6)

g5 <- ggplot(fatPhenoCovarLifestyle2[sex=='Male'],aes(bmiCalc,gfat))+
geom_point()+
geom_line(aes(y=gfat_bmiPred),color='red',size=1)+
geom_line(aes(y=gfat_bmiSplinePred),color='blue',size=1)+
    xlab('BMI (kg/m2)') +
    ylab('GFAT (Liters)') +
    theme_classic() +
#     coord_flip() +
#     scale_x_discrete(limits=rev(levels(riskBarPlotDat$bmiGroup))) +
    theme(axis.title.y = element_text(size=20),
          axis.text.y = element_text(size=18,color='black'),
          axis.ticks.y = element_blank(),
          axis.title.x = element_text(size=20),
          axis.text.x = element_text(size=18,color='black'),
          axis.ticks.x = element_blank(),
          legend.text.align=0,
         legend.key.size=unit(1,'cm'),
         legend.title = element_text(size=16),
         legend.text = element_text(size=16),
         legend.position='none')+
annotate("text",x=20,y=21.5,label=paste0('Male'),size=6)

g6 <- ggplot(fatPhenoCovarLifestyle2[sex=='Female'],aes(bmiCalc,gfat))+
geom_point()+
geom_line(aes(y=gfat_bmiPred),color='red',size=1)+
geom_line(aes(y=gfat_bmiSplinePred),color='blue',size=1)+
    xlab('BMI (kg/m2)') +
    ylab('GFAT (Liters)') +
    theme_classic() +
#     coord_flip() +
#     scale_x_discrete(limits=rev(levels(riskBarPlotDat$bmiGroup))) +
    theme(axis.title.y = element_text(size=20),
          axis.text.y = element_text(size=18,color='black'),
          axis.ticks.y = element_blank(),
          axis.title.x = element_text(size=20),
          axis.text.x = element_text(size=18,color='black'),
          axis.ticks.x = element_blank(),
          legend.text.align=0,
         legend.key.size=unit(1,'cm'),
         legend.title = element_text(size=16),
         legend.text = element_text(size=16),
         legend.position='none')+
annotate("text",x=20,y=25.5,label=paste0('Female'),size=6)

g7 <- (g1|g2)/(g3|g4)/(g5|g6)
g7

# ggsave('./2021.03.11_paperFigs/paper1_MLandEpi/revisions/linear_vs_flexible_bmi_adjustment.pdf',
#        g7,height=15,width=10)

# options(bitmapType='cairo')
# ggsave('./2021.03.11_paperFigs/paper1_MLandEpi/revisions/linear_vs_flexible_bmi_adjustment.png',
#        g7,height=15,width=10)

## Build correlation plots between VAT, ASAT, GFAT and anthropometric variables (1) stratified by sex and (2) stratified by sex and self-reported ethnicity subgroup

In [ ]:
dtforcorrMale <- fatPhenoCovarLifestyle2[sex=='Male',c('vat','asat','gfat',
                                           'bmiCalc','waistcmInstance2',
                                           'hipcmInstance2')]

names(dtforcorrMale) <- c('VAT','ASAT','GFAT',
                          'BMI','Waist','Hip')

M2 <- cor(dtforcorrMale,use = 'complete.obs')

dtforcorrFemale <- fatPhenoCovarLifestyle2[sex=='Female',c('vat','asat','gfat',
                                           'bmiCalc','waistcmInstance2',
                                           'hipcmInstance2')]

names(dtforcorrFemale) <- c('VAT','ASAT','GFAT',
                          'BMI','Waist','Hip')

M3 <- cor(dtforcorrFemale,use = 'complete.obs')

M4 <- copy(M3)
for(i in 1:6){
    for(j in 1:6){
        if(j < i){
            M4[i,j] <- M2[i,j]
        }
    }
}

In [ ]:
# pdf(height=10,width=10,file='/medpop/esp2/sagrawal/fat_distribution/2021.03.11_paperFigs/paper1_MLandEpi/corrPlots/phenoCorr_AIcombined_gfatupdate.pdf')

options(repr.plot.width=10,repr.plot.height=10)
# corrplot(M3,type='upper',order='original',method='color',
#          addCoef.col='black',tl.col='black')

corrplot(M4,type='full',order='original',col=rev(brewer.pal(n=10,name='RdYlBu')),
         addCoef.col='black',tl.col='black',cl.lim=c(0.5,1),is.corr=FALSE,diag=FALSE)

# dev.off()

In [ ]:
#Revision: Get correlation for self-reported ethnicity groups

for(eth in c('White','Black','South Asian','East Asian')){
    print(eth)
    
dtforcorrMale <- na.omit(fatPhenoCovarLifestyle2[ethnicity_group==eth&sex=='Male',c('vat','asat','gfat',
                                           'bmiCalc','waistcmInstance2',
                                           'hipcmInstance2')])
    
    print(paste0('Male: ',nrow(dtforcorrMale)))

names(dtforcorrMale) <- c('VAT','ASAT','GFAT',
                          'BMI','Waist','Hip')

M2 <- cor(dtforcorrMale,use = 'complete.obs')

dtforcorrFemale <- na.omit(fatPhenoCovarLifestyle2[ethnicity_group==eth&sex=='Female',c('vat','asat','gfat',
                                           'bmiCalc','waistcmInstance2',
                                           'hipcmInstance2')])
    
    print(paste0('Female: ',nrow(dtforcorrFemale)))

names(dtforcorrFemale) <- c('VAT','ASAT','GFAT',
                          'BMI','Waist','Hip')

M3 <- cor(dtforcorrFemale,use = 'complete.obs')

M4 <- copy(M3)
for(i in 1:6){
    for(j in 1:6){
        if(j < i){
            M4[i,j] <- M2[i,j]
        }
    }
}
    
M4 <- format(round(M4,2),nsmall=2)
    
assign(paste0('cormat','_',paste(eth,collapse='_')),M4)
    
    
}

cormat_White
cormat_Black
`cormat_East Asian`
`cormat_South Asian`


## Build "Table 1s" in various subgroups including (1) entire studied cohort, (2) among participants who had previously quantified VAT, ASAT, GFAT and those who did not, and (3) among participants who were randomized into development and holdout sets for each of the VAT, ASAT, and GFAT models

In [ ]:
# saaketSumm(fatPhenoCovarLifestyle2)

In [ ]:
fatPhenoCovarLifestyle2[,weightInstance2:=2.2*weightInstance2]
fatPhenoCovarLifestyle2[,heightCombined:=0.394*heightCombined]
fatPhenoCovarLifestyle2$sex <- factor(fatPhenoCovarLifestyle2$sex,levels=c('Male','Female'))

In [ ]:
#Visually inspect normality of all continuous variables

options(repr.plot.width=12,repr.plot.height=8)

library(gridExtra)

continuous_vec <- c('age_instance2','weightInstance2','heightCombined','bmiCalc',
             'waistcmInstance2','hipcmInstance2','WHRInstance2','sbpInstance2','dbpInstance2',
             'vat','asat','gfat')

for(i in 1:length(continuous_vec)){
    
    tempg <- ggplot(data=fatPhenoCovarLifestyle2,aes_string(x=continuous_vec[i]))+
            geom_density()+
    theme_classic() +
    theme(axis.title.y = element_text(size=12),
          axis.text.y = element_blank(),
          axis.ticks.y = element_blank(),
          axis.title.x = element_text(size=18),
          axis.text.x = element_text(size=16,color='black'),
          axis.ticks.x = element_blank(),
          legend.position='none')
    assign(paste0('g',i),
          tempg)
}

grid.arrange(g1,g2,g3,g4,g5,g6,g7,g8,g9,g10,g11,g12,nrow=4,ncol=3)

In [ ]:
tab1vars <- c('age_instance2','ethnicity_group','sbpInstance2','dbpInstance2','smokingStrat',
              'weightInstance2','heightCombined','bmiCalc',
             'waistcmInstance2','hipcmInstance2','WHRInstance2',
             'vat','asat','gfat','prev_T2D','prev_CAD_int')

tab1contvars <- c('age_instance2','weightInstance2','heightCombined','bmiCalc',
             'waistcmInstance2','hipcmInstance2','WHRInstance2','sbpInstance2','dbpInstance2',
             'vat','asat','gfat')

tab1facvars <- c('ethnicity_group','smokingStrat','prev_T2D','prev_CAD_int')

tab1 <- CreateTableOne(vars=tab1vars,factorVars=tab1facvars,strata='sex',data=fatPhenoCovarLifestyle2)
tab1Mat <- print(tab1,nonnormal=tab1contvars,contDigits=1,quote=FALSE,printToggle=FALSE)
# tab1Mat

# write.csv(tab1Mat,
#          './2021.03.11_paperFigs/paper1_MLandEpi/revisions/table1_medians.csv',
#          row.names=TRUE)

In [ ]:
## Read in participant ids for those randomized to development vs. holdout sets
## for VAT, ASAT, and GFAT CNNs.

makeDir <- '/medpop/esp2/sagrawal/fat_distribution/2021.03.11_datafreeze/'

VAT_dev <- fread(paste0(makeDir,'VAT_dev_IDs.csv'))
VAT_ho <- fread(paste0(makeDir,'VAT_ho_IDs.csv'))
VAT_label_id <- unique(c(VAT_dev$x,VAT_ho$x))

ASAT_dev <- fread(paste0(makeDir,'ASAT_dev_IDs.csv'))
ASAT_ho <- fread(paste0(makeDir,'ASAT_ho_IDs.csv'))
ASAT_label_id <- unique(c(ASAT_dev$x,ASAT_ho$x))

GFAT_dev <- fread(paste0(makeDir,'GFAT_dev_IDs.csv'))
GFAT_ho <- fread(paste0(makeDir,'GFAT_ho_IDs.csv'))
GFAT_label_id <- unique(c(GFAT_dev$x,GFAT_ho$x))

In [ ]:
fatPhenoCovarLifestyle2[,VAT_label:=ifelse(eid%in%VAT_label_id,1,0)]
fatPhenoCovarLifestyle2[VAT_label==1,VAT_label_ho:=ifelse(eid%in%VAT_ho$x,1,0)]

fatPhenoCovarLifestyle2[,ASAT_label:=ifelse(eid%in%ASAT_label_id,1,0)]
fatPhenoCovarLifestyle2[ASAT_label==1,ASAT_label_ho:=ifelse(eid%in%ASAT_ho$x,1,0)]

fatPhenoCovarLifestyle2[,GFAT_label:=ifelse(eid%in%GFAT_label_id,1,0)]
fatPhenoCovarLifestyle2[GFAT_label==1,GFAT_label_ho:=ifelse(eid%in%GFAT_ho$x,1,0)]

fatPhenoCovarLifestyle2[,label_strat:=0]
fatPhenoCovarLifestyle2[VAT_label==1&GFAT_label==1,label_strat:=1]
fatPhenoCovarLifestyle2[VAT_label==1&GFAT_label==0,label_strat:=2]
fatPhenoCovarLifestyle2$label_strat <- factor(fatPhenoCovarLifestyle2$label_strat)

In [ ]:
tab1vars <- c('age_instance2','sex','ethnicity_group','sbpInstance2','dbpInstance2','smokingStrat',
              'weightInstance2','heightCombined','bmiCalc',
             'waistcmInstance2','hipcmInstance2','WHRInstance2',
             'vat','asat','gfat','prev_T2D','prev_CAD_int')

tab1contvars <- c('age_instance2','weightInstance2','heightCombined','bmiCalc',
             'waistcmInstance2','hipcmInstance2','WHRInstance2','sbpInstance2','dbpInstance2',
             'vat','asat','gfat')

tab1facvars <- c('ethnicity_group','sex','smokingStrat','prev_T2D','prev_CAD_int')

tab1 <- CreateTableOne(vars=tab1vars,factorVars=tab1facvars,strata='label_strat',data=fatPhenoCovarLifestyle2)
tab1Mat <- print(tab1,nonnormal=tab1contvars,contDigits=1,quote=FALSE,printToggle=FALSE)
# tab1Mat

# write.csv(tab1Mat,
#          './2021.03.11_paperFigs/paper1_MLandEpi/revisions/table1_vat_gfat_label_discrepancy.csv',
#          row.names=TRUE)

In [ ]:
tab1vars <- c('age_instance2','sex','ethnicity_group','sbpInstance2','dbpInstance2','smokingStrat',
              'weightInstance2','heightCombined','bmiCalc',
             'waistcmInstance2','hipcmInstance2','WHRInstance2',
             'vat','asat','gfat','prev_T2D','prev_CAD_int')

tab1contvars <- c('age_instance2','weightInstance2','heightCombined','bmiCalc',
             'waistcmInstance2','hipcmInstance2','WHRInstance2','sbpInstance2','dbpInstance2',
             'vat','asat','gfat')

tab1facvars <- c('ethnicity_group','sex','smokingStrat','prev_T2D','prev_CAD_int')

tab1 <- CreateTableOne(vars=tab1vars,factorVars=tab1facvars,strata='VAT_label_ho',data=fatPhenoCovarLifestyle2)
tab1Mat <- print(tab1,nonnormal=tab1contvars,contDigits=1,quote=FALSE,printToggle=FALSE)
# tab1Mat

# write.csv(tab1Mat,
#          './2021.03.11_paperFigs/paper1_MLandEpi/revisions/table1_vat_dev_ho.csv',
#          row.names=TRUE)

In [ ]:
tab1vars <- c('age_instance2','sex','ethnicity_group','sbpInstance2','dbpInstance2','smokingStrat',
              'weightInstance2','heightCombined','bmiCalc',
             'waistcmInstance2','hipcmInstance2','WHRInstance2',
             'vat','asat','gfat','prev_T2D','prev_CAD_int')

tab1contvars <- c('age_instance2','weightInstance2','heightCombined','bmiCalc',
             'waistcmInstance2','hipcmInstance2','WHRInstance2','sbpInstance2','dbpInstance2',
             'vat','asat','gfat')

tab1facvars <- c('ethnicity_group','sex','smokingStrat','prev_T2D','prev_CAD_int')

tab1 <- CreateTableOne(vars=tab1vars,factorVars=tab1facvars,strata='ASAT_label_ho',data=fatPhenoCovarLifestyle2)
tab1Mat <- print(tab1,nonnormal=tab1contvars,contDigits=1,quote=FALSE,printToggle=FALSE)
# tab1Mat

# write.csv(tab1Mat,
#          './2021.03.11_paperFigs/paper1_MLandEpi/revisions/table1_asat_dev_ho.csv',
#          row.names=TRUE)

In [ ]:
tab1vars <- c('age_instance2','sex','ethnicity_group','sbpInstance2','dbpInstance2','smokingStrat',
              'weightInstance2','heightCombined','bmiCalc',
             'waistcmInstance2','hipcmInstance2','WHRInstance2',
             'vat','asat','gfat','prev_T2D','prev_CAD_int')

tab1contvars <- c('age_instance2','weightInstance2','heightCombined','bmiCalc',
             'waistcmInstance2','hipcmInstance2','WHRInstance2','sbpInstance2','dbpInstance2',
             'vat','asat','gfat')

tab1facvars <- c('ethnicity_group','sex','smokingStrat','prev_T2D','prev_CAD_int')

tab1 <- CreateTableOne(vars=tab1vars,factorVars=tab1facvars,strata='GFAT_label_ho',data=fatPhenoCovarLifestyle2)
tab1Mat <- print(tab1,nonnormal=tab1contvars,contDigits=2,quote=FALSE,printToggle=FALSE)
# tab1Mat

# write.csv(tab1Mat,
#          './2021.03.11_paperFigs/paper1_MLandEpi/revisions/table1_gfat_dev_ho.csv',
#          row.names=TRUE)

## Build VAT, ASAT, and GFAT density plots across (1) sex and BMI subgroups and (2) sex, BMI, and self-reported ethnicity subgroups

In [ ]:
densityPlotDat <- fatPhenoCovarLifestyle2[,c('bmiGroup','sex','vat','asat','gfat')]
densityPlotDat.m1 <- melt(densityPlotDat,id.vars=c('bmiGroup','sex'),
                         measure.vars=c('vat','asat','gfat'))

densityPlotDat.m1[,valMeans:=lapply(.SD,mean),.SDcols=c('value'),by=c('bmiGroup','sex','variable')]
densityPlotDat.m1[,valMedians:=lapply(.SD,median),.SDcols=c('value'),by=c('bmiGroup','sex','variable')]

In [ ]:
densityPlotDat2 <- fatPhenoCovarLifestyle2[,c('bmiGroup','sex','vatadjbmi','asatadjbmi','gfatadjbmi')]
densityPlotDat2.m1 <- melt(densityPlotDat2,id.vars=c('bmiGroup','sex'),
                         measure.vars=c('vatadjbmi','asatadjbmi','gfatadjbmi'))

densityPlotDat2.m1[,valMeans:=lapply(.SD,mean),.SDcols=c('value'),by=c('bmiGroup','sex','variable')]
densityPlotDat2.m1[,valMedians:=lapply(.SD,median),.SDcols=c('value'),by=c('bmiGroup','sex','variable')]

In [ ]:
# options(repr.plot.width=6,repr.plot.height=3)
fatDensityByBmi <- function(bmiGroupChar,sexChar){
    g1 <- ggplot(densityPlotDat.m1[bmiGroup==bmiGroupChar&sex==sexChar],aes(x=value,fill=variable))+
#     geom_histogram(alpha=0.4,position='identity')+
    geom_density(alpha=0.4)+
    geom_vline(aes(xintercept=valMedians,color=variable),linetype='dashed')+
    scale_x_continuous(limits=c(0,25),breaks=seq(0,25,5))+
#     scale_y_continuous(expand=expansion(c(0,0))) +
    xlab('Volume (Liters)') +
    ylab('density') +
    scale_fill_manual(values=c('#e53377','#ecb31f','#27b2af')) +
    scale_color_manual(values=c('#e53377','#ecb31f','#27b2af'))+
    theme_classic() +
    theme(axis.title.y = element_text(size=18),
          axis.text.y = element_text(size=16,color='black'),
          axis.ticks.y = element_blank(),
          axis.title.x = element_text(size=18),
          axis.text.x = element_text(size=16,color='black'),
          axis.ticks.x = element_blank(),
          legend.position='none',
          legend.text.align=0,
         legend.key.size=unit(1,'cm'),
         legend.title = element_text(size=16),
         legend.text = element_text(size=16))
    
    return(g1)
}



In [ ]:
# options(repr.plot.width=6,repr.plot.height=3)
AdjfatDensityByBmi <- function(bmiGroupChar,sexChar){
    g1 <- ggplot(densityPlotDat2.m1[bmiGroup==bmiGroupChar&sex==sexChar],aes(x=value,fill=variable))+
    geom_density(alpha=0.4)+
    geom_vline(aes(xintercept=valMedians,color=variable),linetype='dashed')+
    scale_x_continuous(limits=c(-10,10),breaks=seq(-10,10,4))+
    xlab('Residuals (Liters)') +
    ylab('density') +
    scale_fill_manual(values=c('#e53377','#ecb31f','#27b2af')) +
    scale_color_manual(values=c('#e53377','#ecb31f','#27b2af'))+
    theme_classic() +
    theme(axis.title.y = element_text(size=18),
          axis.text.y = element_text(size=16,color='black'),
          axis.ticks.y = element_blank(),
          axis.title.x = element_text(size=18),
          axis.text.x = element_text(size=16,color='black'),
          axis.ticks.x = element_blank(),
          legend.position='none',
          legend.text.align=0,
         legend.key.size=unit(1,'cm'),
         legend.title = element_text(size=16),
         legend.text = element_text(size=16))
    
    return(g1)
}



In [ ]:
for(char in c('Normal','Overweight','Obese')){
    assign(paste0(char,'_Male'),fatDensityByBmi(char,'Male'))
}

Normal_Male <- Normal_Male +
scale_y_continuous(limits=c(0,0.62),breaks=seq(0,0.6,0.3),expand=expansion(c(0,0))) +
xlab('')

Overweight_Male <- Overweight_Male +
scale_y_continuous(limits=c(0,0.62),breaks=seq(0,0.6,0.3),expand=expansion(c(0,0))) +
xlab('')

Obese_Male <- Obese_Male +
scale_y_continuous(limits=c(0,0.62),breaks=seq(0,0.6,0.3),expand=expansion(c(0,0)))

MalePlotsRaw <- Normal_Male/Overweight_Male/Obese_Male


In [ ]:
for(char in c('Normal','Overweight','Obese')){
    assign(paste0(char,'_Female'),fatDensityByBmi(char,'Female'))
}

Normal_Female <- Normal_Female +
scale_y_continuous(limits=c(0,0.62),breaks=seq(0,0.6,0.3),expand=expansion(c(0,0))) +
xlab('') +
ylab('')

Overweight_Female <- Overweight_Female +
scale_y_continuous(limits=c(0,0.62),breaks=seq(0,0.6,0.3),expand=expansion(c(0,0))) +
xlab('') +
ylab('')

Obese_Female <- Obese_Female +
scale_y_continuous(limits=c(0,0.62),breaks=seq(0,0.6,0.3),expand=expansion(c(0,0))) +
# xlab('') +
ylab('')

FemalePlotsRaw <- Normal_Female/Overweight_Female/Obese_Female


In [ ]:
options(repr.plot.width=8,repr.plot.height=8)
allRaw <- MalePlotsRaw|FemalePlotsRaw
allRaw

# ggsave('/medpop/esp2/sagrawal/fat_distribution/2021.03.11_paperFigs/paper1_MLandEpi/vatAsatGfatAllDensity_gfatupdate.pdf',
#       allRaw,height=8,width=8)

In [ ]:
for(char in c('Normal','Overweight','Obese')){
    assign(paste0(char,'_Male'),AdjfatDensityByBmi(char,'Male'))
}

Normal_Male <- Normal_Male +
scale_y_continuous(limits=c(0,0.65),breaks=seq(0,0.6,0.3),expand=expansion(c(0,0))) +
xlab('')

Overweight_Male <- Overweight_Male +
scale_y_continuous(limits=c(0,0.65),breaks=seq(0,0.6,0.3),expand=expansion(c(0,0))) +
xlab('')

Obese_Male <- Obese_Male +
scale_y_continuous(limits=c(0,0.65),breaks=seq(0,0.6,0.3),expand=expansion(c(0,0)))

MalePlotsAdj <- Normal_Male/Overweight_Male/Obese_Male


In [ ]:
for(char in c('Normal','Overweight','Obese')){
    assign(paste0(char,'_Female'),AdjfatDensityByBmi(char,'Female'))
}

Normal_Female <- Normal_Female +
scale_y_continuous(limits=c(0,0.65),breaks=seq(0,0.6,0.3),expand=expansion(c(0,0))) +
xlab('')+
ylab('')

Overweight_Female <- Overweight_Female +
scale_y_continuous(limits=c(0,0.65),breaks=seq(0,0.6,0.3),expand=expansion(c(0,0))) +
xlab('')+
ylab('')

Obese_Female <- Obese_Female +
scale_y_continuous(limits=c(0,0.65),breaks=seq(0,0.6,0.3),expand=expansion(c(0,0))) +
# xlab('')+
ylab('')

FemalePlotsAdj <- Normal_Female/Overweight_Female/Obese_Female


In [ ]:
options(repr.plot.width=8,repr.plot.height=8)
allAdj <- MalePlotsAdj|FemalePlotsAdj
allAdj

# ggsave('/medpop/esp2/sagrawal/fat_distribution/2021.03.11_paperFigs/paper1_MLandEpi/vatAsatGfatAdjDensity_gfatupdate.pdf',
#       allAdj,height=8,width=8)

### Revision sensitivty analysis: show VAT, ASAT, GFAT distributions by self-reported ethnicity group using alternate BMI cutoffs for obesity reported here: https://pubmed.ncbi.nlm.nih.gov/24974975/

In [ ]:
# options(repr.plot.width=6,repr.plot.height=3)
fatDensityByBmi_joint <- function(dat,bmiGroupChar,sexChar,ethnicityChar){
    g1 <- ggplot(dat[bmiDichotJoint==bmiGroupChar&sex==sexChar&ethnicity_group==ethnicityChar],aes(x=value,fill=variable))+
#     geom_histogram(alpha=0.4,position='identity')+
    geom_density(alpha=0.4)+
    geom_vline(aes(xintercept=valMedians,color=variable),linetype='dashed')+
    scale_x_continuous(limits=c(0,25),breaks=seq(0,20,10))+
#     scale_y_continuous(expand=expansion(c(0,0))) +
    xlab('Volume (Liters)') +
    ylab('density') +
    scale_fill_manual(values=c('#e53377','#ecb31f','#27b2af')) +
    scale_color_manual(values=c('#e53377','#ecb31f','#27b2af'))+
    theme_classic() +
    theme(axis.title.y = element_text(size=16),
          axis.text.y = element_text(size=14,color='black'),
          axis.ticks.y = element_blank(),
          axis.title.x = element_text(size=16),
          axis.text.x = element_text(size=14,color='black'),
          axis.ticks.x = element_blank(),
          legend.position='none',
          legend.text.align=0,
         legend.key.size=unit(1,'cm'),
         legend.title = element_text(size=14),
         legend.text = element_text(size=14))
    
    return(g1)
}

fatDensityByBmi_ethnicity <- function(dat,bmiGroupChar,sexChar,ethnicityChar){
    g1 <- ggplot(dat[bmiDichotEthnicity==bmiGroupChar&sex==sexChar&ethnicity_group==ethnicityChar],aes(x=value,fill=variable))+
#     geom_histogram(alpha=0.4,position='identity')+
    geom_density(alpha=0.4)+
    geom_vline(aes(xintercept=valMedians,color=variable),linetype='dashed')+
    scale_x_continuous(limits=c(0,25),breaks=seq(0,20,10))+
#     scale_y_continuous(expand=expansion(c(0,0))) +
    xlab('Volume (Liters)') +
    ylab('density') +
    scale_fill_manual(values=c('#e53377','#ecb31f','#27b2af')) +
    scale_color_manual(values=c('#e53377','#ecb31f','#27b2af'))+
    theme_classic() +
    theme(axis.title.y = element_text(size=16),
          axis.text.y = element_text(size=14,color='black'),
          axis.ticks.y = element_blank(),
          axis.title.x = element_text(size=16),
          axis.text.x = element_text(size=14,color='black'),
          axis.ticks.x = element_blank(),
          legend.position='none',
          legend.text.align=0,
         legend.key.size=unit(1,'cm'),
         legend.title = element_text(size=14),
         legend.text = element_text(size=14))
    
    return(g1)
}



In [ ]:
densityPlotDat <- fatPhenoCovarLifestyle2[,c('bmiDichotJoint','bmiDichotEthnicity','ethnicity_group','sex','vat','asat','gfat')]

densityPlotDat.m1 <- melt(densityPlotDat,id.vars=c('bmiDichotJoint','ethnicity_group','sex'),
                         measure.vars=c('vat','asat','gfat'))
densityPlotDat.m1[,valMedians:=lapply(.SD,median),.SDcols=c('value'),by=c('bmiDichotJoint','ethnicity_group','sex','variable')]

densityPlotDat.m2 <- melt(densityPlotDat,id.vars=c('bmiDichotEthnicity','ethnicity_group','sex'),
                         measure.vars=c('vat','asat','gfat'))
densityPlotDat.m2[,valMedians:=lapply(.SD,median),.SDcols=c('value'),by=c('bmiDichotEthnicity','ethnicity_group','sex','variable')]


In [ ]:
for(char1 in c('Low','High')){
    for(char2 in c('Black','East Asian','South Asian')){
        for(char3 in c('Male','Female')){
            assign(paste0(gsub(" ","",char2),'_',char1,'_',char3,'_jointplot'),
                  fatDensityByBmi_joint(densityPlotDat.m1,char1,char3,char2))
            
            assign(paste0(gsub(" ","",char2),'_',char1,'_',char3,'_ethnicityplot'),
                  fatDensityByBmi_ethnicity(densityPlotDat.m2,char1,char3,char2))
        }
    }
}

In [ ]:
Black_plots <- (Black_Low_Male_jointplot | Black_Low_Female_jointplot)/
                (Black_High_Male_jointplot | Black_High_Female_jointplot)/
                (Black_Low_Male_ethnicityplot | Black_Low_Female_ethnicityplot)/
                (Black_High_Male_ethnicityplot | Black_High_Female_ethnicityplot)

EastAsian_plots <- (EastAsian_Low_Male_jointplot | EastAsian_Low_Female_jointplot)/
                (EastAsian_High_Male_jointplot | EastAsian_High_Female_jointplot)/
                (EastAsian_Low_Male_ethnicityplot | EastAsian_Low_Female_ethnicityplot)/
                (EastAsian_High_Male_ethnicityplot | EastAsian_High_Female_ethnicityplot)

SouthAsian_plots <- (SouthAsian_Low_Male_jointplot | SouthAsian_Low_Female_jointplot)/
                (SouthAsian_High_Male_jointplot | SouthAsian_High_Female_jointplot)/
                (SouthAsian_Low_Male_ethnicityplot | SouthAsian_Low_Female_ethnicityplot)/
                (SouthAsian_High_Male_ethnicityplot | SouthAsian_High_Female_ethnicityplot)

In [ ]:
options(repr.plot.width=16,repr.plot.height=8)
outplots <- Black_plots | EastAsian_plots | SouthAsian_plots
outplots

# ggsave('/medpop/esp2/sagrawal/fat_distribution/2021.03.11_paperFigs/paper1_MLandEpi/revisions/densityplots_ethnicitybmi.pdf',
#       outplots,height=8,width=16)



# Logistic regression

In [ ]:
#No significance to renaming here
trainDat <- copy(fatPhenoCovarLifestyle2)

In [ ]:
table(trainDat$mriNum,useNA = 'always')

In [ ]:
trainDat[,mriNum_141243:=ifelse(mriNum==141243,1,0)]
trainDat[,mriNum_141631:=ifelse(mriNum==141631,1,0)]

In [ ]:
returnORwrapper <- function(mod,subgroupName){
    sumtempMod <- summary(mod)
    tempCoef <- sumtempMod$coefficients
    
    temprowNames <- row.names(tempCoef)
    bmiCalcIndex <- which(temprowNames=='bmiCalcScaled')
    vatadjbmiIndex <- which(temprowNames=='vatadjbmiScaled')
    asatadjbmiIndex <- which(temprowNames=='asatadjbmiScaled')
    gfatadjbmiIndex <- which(temprowNames=='gfatadjbmiScaled')
    
    ORupperlower <- function(indexVal){
        tempOR <- exp(tempCoef[indexVal,1])
        tempUpper <- exp(tempCoef[indexVal,1]+(1.96*tempCoef[indexVal,2]))
        tempLower <- exp(tempCoef[indexVal,1]-(1.96*tempCoef[indexVal,2]))
        tempORwCI <- paste0(format(round(tempOR,2),nsmall=2),
                            ' ',
                            '(',
                            format(round(tempLower,2),nsmall=2),
                            '-',
                            format(round(tempUpper,2),nsmall=2),
                            ')')
        
        tempPval <- tempCoef[indexVal,4]
        if(tempPval < 0.001){
            tempPval <- formatC(tempPval,format='e',digits=1)
        } else {
            tempPval <- format(round(tempPval,3),nsmall=3)
        }
        
        return(c(tempOR,tempUpper,tempLower,tempORwCI,tempPval))
    }
    
    bmiCalcVec <- ORupperlower(bmiCalcIndex)
    vatadjbmiVec <- ORupperlower(vatadjbmiIndex)
    asatadjbmiVec <- ORupperlower(asatadjbmiIndex)
    gfatadjbmiVec <- ORupperlower(gfatadjbmiIndex)
    
    outDat <- data.table('subgroupName'=subgroupName,'bmiCalc_OR'=bmiCalcVec[1],
                        'bmiCalc_upper'=bmiCalcVec[2], 'bmiCalc_lower'=bmiCalcVec[3],
                         'bmiCalc_ORwCI'=bmiCalcVec[4],'bmiCalc_Pval'=bmiCalcVec[5],
                        'vatadjbmi_OR'=vatadjbmiVec[1], 'vatadjbmi_upper'=vatadjbmiVec[2],
                        'vatadjbmi_lower'=vatadjbmiVec[3], 'vatadjbmi_ORwCI'=vatadjbmiVec[4],
                         'vatadjbmi_Pval'=vatadjbmiVec[5], 'asatadjbmi_OR'=asatadjbmiVec[1],
                        'asatadjbmi_upper'=asatadjbmiVec[2], 'asatadjbmi_lower'=asatadjbmiVec[3],
                         'asatadjbmi_ORwCI'=asatadjbmiVec[4],'asatadjbmi_Pval'=asatadjbmiVec[5],
                        'gfatadjbmi_OR'=gfatadjbmiVec[1], 'gfatadjbmi_upper'=gfatadjbmiVec[2],
                        'gfatadjbmi_lower'=gfatadjbmiVec[3],'gfatadjbmi_ORwCI'=gfatadjbmiVec[4],
                        'gfatadjbmi_Pval'=gfatadjbmiVec[5])
    
    return(outDat)
}

In [ ]:
giveORwCI <- function(mod,var){
    tempCoef <- summary(mod)$coefficients
    temprowNames <- row.names(tempCoef)
    indexVal <- which(temprowNames==var)

    tempOR <- exp(tempCoef[indexVal,1])
    tempUpper <- exp(tempCoef[indexVal,1]+(1.96*tempCoef[indexVal,2]))
    tempLower <- exp(tempCoef[indexVal,1]-(1.96*tempCoef[indexVal,2]))
    tempORwCI <- paste0(format(round(tempOR,2),nsmall=2),
                            ' ',
                            '(',
                            format(round(tempLower,2),nsmall=2),
                            '-',
                            format(round(tempUpper,2),nsmall=2),
                            ')')
    return(tempORwCI)
}


In [ ]:
prev_T2D_all <- glm(prev_T2D~age_instance2+
                      sex+bmiCalcScaled+mriNum_141243+mriNum_141631,
                      data=trainDat,family='binomial')

summary(prev_T2D_all)$coefficients
giveORwCI(prev_T2D_all,'bmiCalcScaled')

prev_T2D_all <- glm(prev_T2D~age_instance2+
                      sex+vatScaled+mriNum_141243+mriNum_141631,
                      data=trainDat,family='binomial')

summary(prev_T2D_all)$coefficients
giveORwCI(prev_T2D_all,'vatScaled')

prev_T2D_all <- glm(prev_T2D~age_instance2+
                      sex+asatScaled+mriNum_141243+mriNum_141631,
                      data=trainDat,family='binomial')

summary(prev_T2D_all)$coefficients
giveORwCI(prev_T2D_all,'asatScaled')

prev_T2D_all <- glm(prev_T2D~age_instance2+
                      sex+gfatScaled+mriNum_141243+mriNum_141631,
                      data=trainDat,family='binomial')

summary(prev_T2D_all)$coefficients
giveORwCI(prev_T2D_all,'gfatScaled')

In [ ]:
prev_CAD_int_all <- glm(prev_CAD_int~age_instance2+
                      sex+bmiCalcScaled+mriNum_141243+mriNum_141631,
                      data=trainDat,family='binomial')

summary(prev_CAD_int_all)$coefficients
giveORwCI(prev_CAD_int_all,'bmiCalcScaled')

prev_CAD_int_all <- glm(prev_CAD_int~age_instance2+
                      sex+vatScaled+mriNum_141243+mriNum_141631,
                      data=trainDat,family='binomial')

summary(prev_CAD_int_all)$coefficients
giveORwCI(prev_CAD_int_all,'vatScaled')

prev_CAD_int_all <- glm(prev_CAD_int~age_instance2+
                      sex+asatScaled+mriNum_141243+mriNum_141631,
                      data=trainDat,family='binomial')

summary(prev_CAD_int_all)$coefficients
giveORwCI(prev_CAD_int_all,'asatScaled')

prev_CAD_int_all <- glm(prev_CAD_int~age_instance2+
                      sex+gfatScaled+mriNum_141243+mriNum_141631,
                      data=trainDat,family='binomial')

summary(prev_CAD_int_all)$coefficients
giveORwCI(prev_CAD_int_all,'gfatScaled')

# Revision: Sensitivity analysis with adjustment for additional covariates

In [ ]:
prev_T2D_all_original <- glm(prev_T2D~age_instance2+
                      sex+bmiCalcScaled+vatadjbmiScaled+asatadjbmiScaled+gfatadjbmiScaled+mriNum_141243+mriNum_141631,
                      data=trainDat,family='binomial')

# summary(prev_T2D_all_original)

prev_T2D_all_Snadjust1 <- glm(prev_T2D~age_instance2+
                      sex+bmiCalcScaled+vatadjbmiScaled+asatadjbmiScaled+gfatadjbmiScaled+mriNum_141243+mriNum_141631+weightInstance2+heightInstance2+smokingStrat+ethnicity_group,
                      data=trainDat,family='binomial')

# summary(prev_T2D_all_Snadjust1)

T2D_out <- data.table()
T2D_out <- rbind(T2D_out,returnORwrapper(prev_T2D_all_original,'Model1'))
T2D_out <- rbind(T2D_out,returnORwrapper(prev_T2D_all_Snadjust1,'Model2'))

T2D_out <- T2D_out[,c('subgroupName',
                      'vatadjbmi_ORwCI','vatadjbmi_Pval',
                      'asatadjbmi_ORwCI','asatadjbmi_Pval',
                      'gfatadjbmi_ORwCI','gfatadjbmi_Pval')]

T2D_out

# write.csv(T2D_out,
#          './2021.03.11_paperFigs/paper1_MLandEpi/revisions/prev_t2d_model_sn.csv',
#          row.names=FALSE)


In [ ]:
prev_CAD_int_all_original <- glm(prev_CAD_int~age_instance2+
                      sex+bmiCalcScaled+vatadjbmiScaled+asatadjbmiScaled+gfatadjbmiScaled+mriNum_141243+mriNum_141631,
                      data=trainDat,family='binomial')

# summary(prev_CAD_int_all_original)



prev_CAD_int_all_Snadjust1 <- glm(prev_CAD_int~age_instance2+
                      sex+bmiCalcScaled+vatadjbmiScaled+asatadjbmiScaled+gfatadjbmiScaled+mriNum_141243+mriNum_141631+weightInstance2+heightInstance2+smokingStrat+ethnicity_group,
                      data=trainDat,family='binomial')

# summary(prev_CAD_int_all_Snadjust1)


prev_CAD_int_all_Snadjust2 <- glm(prev_CAD_int~age_instance2+
                      sex+bmiCalcScaled+vatadjbmiScaled+asatadjbmiScaled+gfatadjbmiScaled+mriNum_141243+mriNum_141631+prev_T2D,
                      data=trainDat,family='binomial')

# summary(prev_CAD_int_all_Snadjust2)

CAD_int_out <- data.table()
CAD_int_out <- rbind(CAD_int_out,returnORwrapper(prev_CAD_int_all_original,'Model1'))
CAD_int_out <- rbind(CAD_int_out,returnORwrapper(prev_CAD_int_all_Snadjust1,'Model2'))
CAD_int_out <- rbind(CAD_int_out,returnORwrapper(prev_CAD_int_all_Snadjust2,'Model3'))

CAD_int_out <- CAD_int_out[,c('subgroupName',
                      'vatadjbmi_ORwCI','vatadjbmi_Pval',
                      'asatadjbmi_ORwCI','asatadjbmi_Pval',
                      'gfatadjbmi_ORwCI','gfatadjbmi_Pval')]

CAD_int_out

# write.csv(CAD_int_out,
#          './2021.03.11_paperFigs/paper1_MLandEpi/revisions/prev_cad_model_sn.csv',
#          row.names=FALSE)



## Revision: Explicitly modeling interaction term with BMI

In [ ]:
prev_T2D_all_original <- glm(prev_T2D~age_instance2+
                      sex+bmiCalcScaled*(vatadjbmiScaled+asatadjbmiScaled+gfatadjbmiScaled)+mriNum_141243+mriNum_141631,
                      data=trainDat,family='binomial')

summary(prev_T2D_all_original)

prev_CAD_all_original <- glm(prev_CAD_int~age_instance2+
                      sex+bmiCalcScaled*(vatadjbmiScaled+asatadjbmiScaled+gfatadjbmiScaled)+mriNum_141243+mriNum_141631,
                      data=trainDat,family='binomial')

summary(prev_CAD_all_original)

In [ ]:
#### Entire cohort ####

prev_T2D_all <- glm(prev_T2D~age_instance2+
                      sex+bmiCalcScaled+vatadjbmiScaled+asatadjbmiScaled+gfatadjbmiScaled+mriNum_141243+mriNum_141631,
                      data=trainDat,family='binomial')

summary(prev_T2D_all)

#### Age subgroups ####

prev_T2D_under60 <- glm(prev_T2D~age_instance2+
                      sex+bmiCalcScaled+vatadjbmiScaled+asatadjbmiScaled+gfatadjbmiScaled+mriNum_141243+mriNum_141631,
                      data=trainDat[ageGroup=='Under 60'],family='binomial')

summary(prev_T2D_under60)


prev_T2D_60to70 <- glm(prev_T2D~age_instance2+
                      sex+bmiCalcScaled+vatadjbmiScaled+asatadjbmiScaled+gfatadjbmiScaled+mriNum_141243+mriNum_141631,
                      data=trainDat[ageGroup=='60 to 70'],family='binomial')

summary(prev_T2D_60to70)


prev_T2D_over70 <- glm(prev_T2D~age_instance2+
                      sex+bmiCalcScaled+vatadjbmiScaled+asatadjbmiScaled+gfatadjbmiScaled+mriNum_141243+mriNum_141631,
                      data=trainDat[ageGroup=='Over 70'],family='binomial')

summary(prev_T2D_over70)

#### Sex subgroups #### (sex covariate removed)

prev_T2D_male <- glm(prev_T2D~age_instance2+
                      bmiCalcScaled+vatadjbmiScaled+asatadjbmiScaled+gfatadjbmiScaled+mriNum_141243+mriNum_141631,
                      data=trainDat[sex=='Male'],family='binomial')

summary(prev_T2D_male)


prev_T2D_female <- glm(prev_T2D~age_instance2+
                      bmiCalcScaled+vatadjbmiScaled+asatadjbmiScaled+gfatadjbmiScaled+mriNum_141243+mriNum_141631,
                      data=trainDat[sex=='Female'],family='binomial')

summary(prev_T2D_female)

#### BMI subgroups ####

prev_T2D_bmiNormal <- glm(prev_T2D~age_instance2+
                      sex+bmiCalcScaled+vatadjbmiScaled+asatadjbmiScaled+gfatadjbmiScaled+mriNum_141243+mriNum_141631,
                      data=trainDat[bmiGroup=='Normal'],family='binomial')

summary(prev_T2D_bmiNormal)


prev_T2D_bmiOverweight <- glm(prev_T2D~age_instance2+
                      sex+bmiCalcScaled+vatadjbmiScaled+asatadjbmiScaled+gfatadjbmiScaled+mriNum_141243+mriNum_141631,
                      data=trainDat[bmiGroup=='Overweight'],family='binomial')

summary(prev_T2D_bmiOverweight)


prev_T2D_bmiObese <- glm(prev_T2D~age_instance2+
                      sex+bmiCalcScaled+vatadjbmiScaled+asatadjbmiScaled+gfatadjbmiScaled+mriNum_141243+mriNum_141631,
                      data=trainDat[bmiGroup=='Obese'],family='binomial')

summary(prev_T2D_bmiObese)

#### Ethnic subgroups ####

prev_T2D_white <- glm(prev_T2D~age_instance2+
                      sex+bmiCalcScaled+vatadjbmiScaled+asatadjbmiScaled+gfatadjbmiScaled+mriNum_141243+mriNum_141631,
                      data=trainDat[ethnicity_group=='White'],family='binomial')

summary(prev_T2D_white)


prev_T2D_black <- glm(prev_T2D~age_instance2+
                      sex+bmiCalcScaled+vatadjbmiScaled+asatadjbmiScaled+gfatadjbmiScaled+mriNum_141243+mriNum_141631,
                      data=trainDat[ethnicity_group=='Black'],family='binomial')

summary(prev_T2D_black)


prev_T2D_eastAsian <- glm(prev_T2D~age_instance2+
                      sex+bmiCalcScaled+vatadjbmiScaled+asatadjbmiScaled+gfatadjbmiScaled+mriNum_141243+mriNum_141631,
                      data=trainDat[ethnicity_group=='East Asian'],family='binomial')

summary(prev_T2D_eastAsian)


prev_T2D_southAsian <- glm(prev_T2D~age_instance2+
                      sex+bmiCalcScaled+vatadjbmiScaled+asatadjbmiScaled+gfatadjbmiScaled+mriNum_141243+mriNum_141631,
                      data=trainDat[ethnicity_group=='South Asian'],family='binomial')

summary(prev_T2D_southAsian)


prev_T2D_other <- glm(prev_T2D~age_instance2+
                      sex+bmiCalcScaled+vatadjbmiScaled+asatadjbmiScaled+gfatadjbmiScaled+mriNum_141243+mriNum_141631,
                      data=trainDat[ethnicity_group=='Other'],family='binomial')

summary(prev_T2D_other)


In [ ]:
prev_T2D_dataOR <- data.table()
prev_T2D_dataOR <- rbind(prev_T2D_dataOR,returnORwrapper(prev_T2D_all,'Entire Cohort'))

prev_T2D_dataOR <- rbind(prev_T2D_dataOR,returnORwrapper(prev_T2D_under60,'Under 60'))
prev_T2D_dataOR <- rbind(prev_T2D_dataOR,returnORwrapper(prev_T2D_60to70,'60 to 70'))
prev_T2D_dataOR <- rbind(prev_T2D_dataOR,returnORwrapper(prev_T2D_bmiObese,'Over 70'))

prev_T2D_dataOR <- rbind(prev_T2D_dataOR,returnORwrapper(prev_T2D_male,'Male'))
prev_T2D_dataOR <- rbind(prev_T2D_dataOR,returnORwrapper(prev_T2D_female,'Female'))

prev_T2D_dataOR <- rbind(prev_T2D_dataOR,returnORwrapper(prev_T2D_bmiNormal,'Under 25'))
prev_T2D_dataOR <- rbind(prev_T2D_dataOR,returnORwrapper(prev_T2D_bmiOverweight,'25 to 30'))
prev_T2D_dataOR <- rbind(prev_T2D_dataOR,returnORwrapper(prev_T2D_bmiObese,'Over 30'))

prev_T2D_dataOR <- rbind(prev_T2D_dataOR,returnORwrapper(prev_T2D_white,'White'))
prev_T2D_dataOR <- rbind(prev_T2D_dataOR,returnORwrapper(prev_T2D_black,'Black'))
prev_T2D_dataOR <- rbind(prev_T2D_dataOR,returnORwrapper(prev_T2D_eastAsian,'East Asian'))
prev_T2D_dataOR <- rbind(prev_T2D_dataOR,returnORwrapper(prev_T2D_southAsian,'South Asian'))
prev_T2D_dataOR <- rbind(prev_T2D_dataOR,returnORwrapper(prev_T2D_other,'Other'))

In [ ]:
# nrow(trainDat)

prev_T2D_all_Nall <- nrow(trainDat[!is.na(bmiCalcScaled)])
prev_T2D_all_Ncase <- nrow(trainDat[!is.na(bmiCalcScaled) & prev_T2D==1])

prev_T2D_under60_Nall <- nrow(trainDat[!is.na(bmiCalcScaled) & ageGroup=='Under 60'])
prev_T2D_under60_Ncase <- nrow(trainDat[!is.na(bmiCalcScaled) & prev_T2D==1 & ageGroup=='Under 60'])

prev_T2D_60to70_Nall <- nrow(trainDat[!is.na(bmiCalcScaled) & ageGroup=='60 to 70'])
prev_T2D_60to70_Ncase <- nrow(trainDat[!is.na(bmiCalcScaled) & prev_T2D==1 & ageGroup=='60 to 70'])

prev_T2D_over70_Nall <- nrow(trainDat[!is.na(bmiCalcScaled) & ageGroup=='Over 70'])
prev_T2D_over70_Ncase <- nrow(trainDat[!is.na(bmiCalcScaled) & prev_T2D==1 & ageGroup=='Over 70'])

prev_T2D_male_Nall <- nrow(trainDat[!is.na(bmiCalcScaled) & sex=='Male'])
prev_T2D_male_Ncase <- nrow(trainDat[!is.na(bmiCalcScaled) & prev_T2D==1 & sex=='Male'])

prev_T2D_female_Nall <- nrow(trainDat[!is.na(bmiCalcScaled) & sex=='Female'])
prev_T2D_female_Ncase <- nrow(trainDat[!is.na(bmiCalcScaled) & prev_T2D==1 & sex=='Female'])

prev_T2D_bmiNormal_Nall <- nrow(trainDat[!is.na(bmiCalcScaled) & bmiGroup=='Normal'])
prev_T2D_bmiNormal_Ncase <- nrow(trainDat[!is.na(bmiCalcScaled) & prev_T2D==1 & bmiGroup=='Normal'])

prev_T2D_bmiOverweight_Nall <- nrow(trainDat[!is.na(bmiCalcScaled) & bmiGroup=='Overweight'])
prev_T2D_bmiOverweight_Ncase <- nrow(trainDat[!is.na(bmiCalcScaled) & prev_T2D==1 & bmiGroup=='Overweight'])

prev_T2D_bmiObese_Nall <- nrow(trainDat[!is.na(bmiCalcScaled) & bmiGroup=='Obese'])
prev_T2D_bmiObese_Ncase <- nrow(trainDat[!is.na(bmiCalcScaled) & prev_T2D==1 & bmiGroup=='Obese'])

prev_T2D_white_Nall <- nrow(trainDat[!is.na(bmiCalcScaled) & ethnicity_group=='White'])
prev_T2D_white_Ncase <- nrow(trainDat[!is.na(bmiCalcScaled) & prev_T2D==1 & ethnicity_group=='White'])

prev_T2D_black_Nall <- nrow(trainDat[!is.na(bmiCalcScaled) & ethnicity_group=='Black'])
prev_T2D_black_Ncase <- nrow(trainDat[!is.na(bmiCalcScaled) & prev_T2D==1 & ethnicity_group=='Black'])

prev_T2D_eastAsian_Nall <- nrow(trainDat[!is.na(bmiCalcScaled) & ethnicity_group=='East Asian'])
prev_T2D_eastAsian_Ncase <- nrow(trainDat[!is.na(bmiCalcScaled) & prev_T2D==1 & ethnicity_group=='East Asian'])

prev_T2D_southAsian_Nall <- nrow(trainDat[!is.na(bmiCalcScaled) & ethnicity_group=='South Asian'])
prev_T2D_southAsian_Ncase <- nrow(trainDat[!is.na(bmiCalcScaled) & prev_T2D==1 & ethnicity_group=='South Asian'])

prev_T2D_other_Nall <- nrow(trainDat[!is.na(bmiCalcScaled) & ethnicity_group=='Other'])
prev_T2D_other_Ncase <- nrow(trainDat[!is.na(bmiCalcScaled) & prev_T2D==1 & ethnicity_group=='Other'])

prev_T2D_dataOR$Nall <- c(prev_T2D_all_Nall,prev_T2D_under60_Nall,prev_T2D_60to70_Nall,
                         prev_T2D_over70_Nall,prev_T2D_male_Nall,prev_T2D_female_Nall,
                         prev_T2D_bmiNormal_Nall,prev_T2D_bmiOverweight_Nall,prev_T2D_bmiObese_Nall,
                         prev_T2D_white_Nall,prev_T2D_black_Nall,
                         prev_T2D_eastAsian_Nall,prev_T2D_southAsian_Nall,prev_T2D_other_Nall)

prev_T2D_dataOR$Ncase <- c(prev_T2D_all_Ncase,prev_T2D_under60_Ncase,prev_T2D_60to70_Ncase,
                         prev_T2D_over70_Ncase,prev_T2D_male_Ncase,prev_T2D_female_Ncase,
                         prev_T2D_bmiNormal_Ncase,prev_T2D_bmiOverweight_Ncase,prev_T2D_bmiObese_Ncase,
                         prev_T2D_white_Ncase,prev_T2D_black_Ncase,
                         prev_T2D_eastAsian_Ncase,prev_T2D_southAsian_Ncase,prev_T2D_other_Ncase)

In [ ]:
prev_T2D_dataOR$Nfrac<-paste0(prev_T2D_dataOR$Ncase,'/',prev_T2D_dataOR$Nall)
prev_T2D_dataOR$Ndec <- format(round(prev_T2D_dataOR$Ncase/prev_T2D_dataOR$Nall*100,1),nsmall=1,trim=TRUE)
prev_T2D_dataOR$Nfracdec <- paste0(prev_T2D_dataOR$Nfrac,' (',prev_T2D_dataOR$Ndec,'%)')

prev_T2D_dataOR$bmiCalc_OR <- as.numeric(prev_T2D_dataOR$bmiCalc_OR)
prev_T2D_dataOR$bmiCalc_upper <- as.numeric(prev_T2D_dataOR$bmiCalc_upper)
prev_T2D_dataOR$bmiCalc_lower <- as.numeric(prev_T2D_dataOR$bmiCalc_lower)

prev_T2D_dataOR$vatadjbmi_OR <- as.numeric(prev_T2D_dataOR$vatadjbmi_OR)
prev_T2D_dataOR$vatadjbmi_upper <- as.numeric(prev_T2D_dataOR$vatadjbmi_upper)
prev_T2D_dataOR$vatadjbmi_lower <- as.numeric(prev_T2D_dataOR$vatadjbmi_lower)

prev_T2D_dataOR$asatadjbmi_OR <- as.numeric(prev_T2D_dataOR$asatadjbmi_OR)
prev_T2D_dataOR$asatadjbmi_upper <- as.numeric(prev_T2D_dataOR$asatadjbmi_upper)
prev_T2D_dataOR$asatadjbmi_lower <- as.numeric(prev_T2D_dataOR$asatadjbmi_lower)

prev_T2D_dataOR$gfatadjbmi_OR <- as.numeric(prev_T2D_dataOR$gfatadjbmi_OR)
prev_T2D_dataOR$gfatadjbmi_upper <- as.numeric(prev_T2D_dataOR$gfatadjbmi_upper)
prev_T2D_dataOR$gfatadjbmi_lower <- as.numeric(prev_T2D_dataOR$gfatadjbmi_lower)

prev_T2D_dataOR

prev_T2D_dataOR_print <- prev_T2D_dataOR[,c('subgroupName','Nfracdec',
                      'vatadjbmi_ORwCI','vatadjbmi_Pval',
                      'asatadjbmi_ORwCI','asatadjbmi_Pval',
                      'gfatadjbmi_ORwCI','gfatadjbmi_Pval')]

# write.csv(prev_T2D_dataOR_print,
#           './2021.03.11_paperFigs/paper1_MLandEpi/revisions/prev_t2d_subgroups.csv',
#           row.names=FALSE)

In [ ]:
#### Entire cohort ####

prev_CAD_int_all <- glm(prev_CAD_int~age_instance2+
                      sex+bmiCalcScaled+vatadjbmiScaled+asatadjbmiScaled+gfatadjbmiScaled+mriNum_141243+mriNum_141631,
                      data=trainDat,family='binomial')

summary(prev_CAD_int_all)

#### Age subgroups ####

prev_CAD_int_under60 <- glm(prev_CAD_int~age_instance2+
                      sex+bmiCalcScaled+vatadjbmiScaled+asatadjbmiScaled+gfatadjbmiScaled+mriNum_141243+mriNum_141631,
                      data=trainDat[ageGroup=='Under 60'],family='binomial')

summary(prev_CAD_int_under60)


prev_CAD_int_60to70 <- glm(prev_CAD_int~age_instance2+
                      sex+bmiCalcScaled+vatadjbmiScaled+asatadjbmiScaled+gfatadjbmiScaled+mriNum_141243+mriNum_141631,
                      data=trainDat[ageGroup=='60 to 70'],family='binomial')

summary(prev_CAD_int_60to70)


prev_CAD_int_over70 <- glm(prev_CAD_int~age_instance2+
                      sex+bmiCalcScaled+vatadjbmiScaled+asatadjbmiScaled+gfatadjbmiScaled+mriNum_141243+mriNum_141631,
                      data=trainDat[ageGroup=='Over 70'],family='binomial')

summary(prev_CAD_int_over70)

#### Sex subgroups #### (sex covariate removed)

prev_CAD_int_male <- glm(prev_CAD_int~age_instance2+
                      bmiCalcScaled+vatadjbmiScaled+asatadjbmiScaled+gfatadjbmiScaled+mriNum_141243+mriNum_141631,
                      data=trainDat[sex=='Male'],family='binomial')

summary(prev_CAD_int_male)


prev_CAD_int_female <- glm(prev_CAD_int~age_instance2+
                      bmiCalcScaled+vatadjbmiScaled+asatadjbmiScaled+gfatadjbmiScaled+mriNum_141243+mriNum_141631,
                      data=trainDat[sex=='Female'],family='binomial')

summary(prev_CAD_int_female)

#### BMI subgroups ####

prev_CAD_int_bmiNormal <- glm(prev_CAD_int~age_instance2+
                      sex+bmiCalcScaled+vatadjbmiScaled+asatadjbmiScaled+gfatadjbmiScaled+mriNum_141243+mriNum_141631,
                      data=trainDat[bmiGroup=='Normal'],family='binomial')

summary(prev_CAD_int_bmiNormal)


prev_CAD_int_bmiOverweight <- glm(prev_CAD_int~age_instance2+
                      sex+bmiCalcScaled+vatadjbmiScaled+asatadjbmiScaled+gfatadjbmiScaled+mriNum_141243+mriNum_141631,
                      data=trainDat[bmiGroup=='Overweight'],family='binomial')

summary(prev_CAD_int_bmiOverweight)


prev_CAD_int_bmiObese <- glm(prev_CAD_int~age_instance2+
                      sex+bmiCalcScaled+vatadjbmiScaled+asatadjbmiScaled+gfatadjbmiScaled+mriNum_141243+mriNum_141631,
                      data=trainDat[bmiGroup=='Obese'],family='binomial')

summary(prev_CAD_int_bmiObese)

#### Ethnic subgroups ####

prev_CAD_int_white <- glm(prev_CAD_int~age_instance2+
                      sex+bmiCalcScaled+vatadjbmiScaled+asatadjbmiScaled+gfatadjbmiScaled+mriNum_141243+mriNum_141631,
                      data=trainDat[ethnicity_group=='White'],family='binomial')

summary(prev_CAD_int_white)


prev_CAD_int_black <- glm(prev_CAD_int~age_instance2+
                      sex+bmiCalcScaled+vatadjbmiScaled+asatadjbmiScaled+gfatadjbmiScaled+mriNum_141243+mriNum_141631,
                      data=trainDat[ethnicity_group=='Black'],family='binomial')

summary(prev_CAD_int_black)


prev_CAD_int_eastAsian <- glm(prev_CAD_int~age_instance2+
                      sex+bmiCalcScaled+vatadjbmiScaled+asatadjbmiScaled+gfatadjbmiScaled+mriNum_141243+mriNum_141631,
                      data=trainDat[ethnicity_group=='East Asian'],family='binomial')

summary(prev_CAD_int_eastAsian)


prev_CAD_int_southAsian <- glm(prev_CAD_int~age_instance2+
                      sex+bmiCalcScaled+vatadjbmiScaled+asatadjbmiScaled+gfatadjbmiScaled+mriNum_141243+mriNum_141631,
                      data=trainDat[ethnicity_group=='South Asian'],family='binomial')

summary(prev_CAD_int_southAsian)


prev_CAD_int_other <- glm(prev_CAD_int~age_instance2+
                      sex+bmiCalcScaled+vatadjbmiScaled+asatadjbmiScaled+gfatadjbmiScaled+mriNum_141243+mriNum_141631,
                      data=trainDat[ethnicity_group=='Other'],family='binomial')

summary(prev_CAD_int_other)



In [ ]:
prev_CAD_int_dataOR <- data.table()
prev_CAD_int_dataOR <- rbind(prev_CAD_int_dataOR,returnORwrapper(prev_CAD_int_all,'Entire Cohort'))

prev_CAD_int_dataOR <- rbind(prev_CAD_int_dataOR,returnORwrapper(prev_CAD_int_under60,'Under 60'))
prev_CAD_int_dataOR <- rbind(prev_CAD_int_dataOR,returnORwrapper(prev_CAD_int_60to70,'60 to 70'))
prev_CAD_int_dataOR <- rbind(prev_CAD_int_dataOR,returnORwrapper(prev_CAD_int_bmiObese,'Over 70'))

prev_CAD_int_dataOR <- rbind(prev_CAD_int_dataOR,returnORwrapper(prev_CAD_int_male,'Male'))
prev_CAD_int_dataOR <- rbind(prev_CAD_int_dataOR,returnORwrapper(prev_CAD_int_female,'Female'))

prev_CAD_int_dataOR <- rbind(prev_CAD_int_dataOR,returnORwrapper(prev_CAD_int_bmiNormal,'Under 25'))
prev_CAD_int_dataOR <- rbind(prev_CAD_int_dataOR,returnORwrapper(prev_CAD_int_bmiOverweight,'25 to 30'))
prev_CAD_int_dataOR <- rbind(prev_CAD_int_dataOR,returnORwrapper(prev_CAD_int_bmiObese,'Over 30'))

prev_CAD_int_dataOR <- rbind(prev_CAD_int_dataOR,returnORwrapper(prev_CAD_int_white,'White'))
prev_CAD_int_dataOR <- rbind(prev_CAD_int_dataOR,returnORwrapper(prev_CAD_int_black,'Black'))
prev_CAD_int_dataOR <- rbind(prev_CAD_int_dataOR,returnORwrapper(prev_CAD_int_eastAsian,'East Asian'))
prev_CAD_int_dataOR <- rbind(prev_CAD_int_dataOR,returnORwrapper(prev_CAD_int_southAsian,'South Asian'))
prev_CAD_int_dataOR <- rbind(prev_CAD_int_dataOR,returnORwrapper(prev_CAD_int_other,'Other'))

In [ ]:
# nrow(trainDat)

prev_CAD_int_all_Nall <- nrow(trainDat[!is.na(bmiCalcScaled)])
prev_CAD_int_all_Ncase <- nrow(trainDat[!is.na(bmiCalcScaled) & prev_CAD_int==1])

prev_CAD_int_under60_Nall <- nrow(trainDat[!is.na(bmiCalcScaled) & ageGroup=='Under 60'])
prev_CAD_int_under60_Ncase <- nrow(trainDat[!is.na(bmiCalcScaled) & prev_CAD_int==1 & ageGroup=='Under 60'])

prev_CAD_int_60to70_Nall <- nrow(trainDat[!is.na(bmiCalcScaled) & ageGroup=='60 to 70'])
prev_CAD_int_60to70_Ncase <- nrow(trainDat[!is.na(bmiCalcScaled) & prev_CAD_int==1 & ageGroup=='60 to 70'])

prev_CAD_int_over70_Nall <- nrow(trainDat[!is.na(bmiCalcScaled) & ageGroup=='Over 70'])
prev_CAD_int_over70_Ncase <- nrow(trainDat[!is.na(bmiCalcScaled) & prev_CAD_int==1 & ageGroup=='Over 70'])

prev_CAD_int_male_Nall <- nrow(trainDat[!is.na(bmiCalcScaled) & sex=='Male'])
prev_CAD_int_male_Ncase <- nrow(trainDat[!is.na(bmiCalcScaled) & prev_CAD_int==1 & sex=='Male'])

prev_CAD_int_female_Nall <- nrow(trainDat[!is.na(bmiCalcScaled) & sex=='Female'])
prev_CAD_int_female_Ncase <- nrow(trainDat[!is.na(bmiCalcScaled) & prev_CAD_int==1 & sex=='Female'])

prev_CAD_int_bmiNormal_Nall <- nrow(trainDat[!is.na(bmiCalcScaled) & bmiGroup=='Normal'])
prev_CAD_int_bmiNormal_Ncase <- nrow(trainDat[!is.na(bmiCalcScaled) & prev_CAD_int==1 & bmiGroup=='Normal'])

prev_CAD_int_bmiOverweight_Nall <- nrow(trainDat[!is.na(bmiCalcScaled) & bmiGroup=='Overweight'])
prev_CAD_int_bmiOverweight_Ncase <- nrow(trainDat[!is.na(bmiCalcScaled) & prev_CAD_int==1 & bmiGroup=='Overweight'])

prev_CAD_int_bmiObese_Nall <- nrow(trainDat[!is.na(bmiCalcScaled) & bmiGroup=='Obese'])
prev_CAD_int_bmiObese_Ncase <- nrow(trainDat[!is.na(bmiCalcScaled) & prev_CAD_int==1 & bmiGroup=='Obese'])

prev_CAD_int_white_Nall <- nrow(trainDat[!is.na(bmiCalcScaled) & ethnicity_group=='White'])
prev_CAD_int_white_Ncase <- nrow(trainDat[!is.na(bmiCalcScaled) & prev_CAD_int==1 & ethnicity_group=='White'])

prev_CAD_int_black_Nall <- nrow(trainDat[!is.na(bmiCalcScaled) & ethnicity_group=='Black'])
prev_CAD_int_black_Ncase <- nrow(trainDat[!is.na(bmiCalcScaled) & prev_CAD_int==1 & ethnicity_group=='Black'])

prev_CAD_int_eastAsian_Nall <- nrow(trainDat[!is.na(bmiCalcScaled) & ethnicity_group=='East Asian'])
prev_CAD_int_eastAsian_Ncase <- nrow(trainDat[!is.na(bmiCalcScaled) & prev_CAD_int==1 & ethnicity_group=='East Asian'])

prev_CAD_int_southAsian_Nall <- nrow(trainDat[!is.na(bmiCalcScaled) & ethnicity_group=='South Asian'])
prev_CAD_int_southAsian_Ncase <- nrow(trainDat[!is.na(bmiCalcScaled) & prev_CAD_int==1 & ethnicity_group=='South Asian'])

prev_CAD_int_other_Nall <- nrow(trainDat[!is.na(bmiCalcScaled) & ethnicity_group=='Other'])
prev_CAD_int_other_Ncase <- nrow(trainDat[!is.na(bmiCalcScaled) & prev_CAD_int==1 & ethnicity_group=='Other'])

prev_CAD_int_dataOR$Nall <- c(prev_CAD_int_all_Nall,prev_CAD_int_under60_Nall,prev_CAD_int_60to70_Nall,
                         prev_CAD_int_over70_Nall,prev_CAD_int_male_Nall,prev_CAD_int_female_Nall,
                         prev_CAD_int_bmiNormal_Nall,prev_CAD_int_bmiOverweight_Nall,prev_CAD_int_bmiObese_Nall,
                         prev_CAD_int_white_Nall,prev_CAD_int_black_Nall,
                         prev_CAD_int_eastAsian_Nall,prev_CAD_int_southAsian_Nall,prev_CAD_int_other_Nall)

prev_CAD_int_dataOR$Ncase <- c(prev_CAD_int_all_Ncase,prev_CAD_int_under60_Ncase,prev_CAD_int_60to70_Ncase,
                         prev_CAD_int_over70_Ncase,prev_CAD_int_male_Ncase,prev_CAD_int_female_Ncase,
                         prev_CAD_int_bmiNormal_Ncase,prev_CAD_int_bmiOverweight_Ncase,prev_CAD_int_bmiObese_Ncase,
                         prev_CAD_int_white_Ncase,prev_CAD_int_black_Ncase,
                         prev_CAD_int_eastAsian_Ncase,prev_CAD_int_southAsian_Ncase,prev_CAD_int_other_Ncase)

In [ ]:
prev_CAD_int_dataOR$Nfrac<-paste0(prev_CAD_int_dataOR$Ncase,'/',prev_CAD_int_dataOR$Nall)
prev_CAD_int_dataOR$Ndec <- format(round(prev_CAD_int_dataOR$Ncase/prev_CAD_int_dataOR$Nall*100,1),nsmall=1,trim=TRUE)
prev_CAD_int_dataOR$Nfracdec <- paste0(prev_CAD_int_dataOR$Nfrac,' (',prev_CAD_int_dataOR$Ndec,'%)')

prev_CAD_int_dataOR$bmiCalc_OR <- as.numeric(prev_CAD_int_dataOR$bmiCalc_OR)
prev_CAD_int_dataOR$bmiCalc_upper <- as.numeric(prev_CAD_int_dataOR$bmiCalc_upper)
prev_CAD_int_dataOR$bmiCalc_lower <- as.numeric(prev_CAD_int_dataOR$bmiCalc_lower)

prev_CAD_int_dataOR$vatadjbmi_OR <- as.numeric(prev_CAD_int_dataOR$vatadjbmi_OR)
prev_CAD_int_dataOR$vatadjbmi_upper <- as.numeric(prev_CAD_int_dataOR$vatadjbmi_upper)
prev_CAD_int_dataOR$vatadjbmi_lower <- as.numeric(prev_CAD_int_dataOR$vatadjbmi_lower)

prev_CAD_int_dataOR$asatadjbmi_OR <- as.numeric(prev_CAD_int_dataOR$asatadjbmi_OR)
prev_CAD_int_dataOR$asatadjbmi_upper <- as.numeric(prev_CAD_int_dataOR$asatadjbmi_upper)
prev_CAD_int_dataOR$asatadjbmi_lower <- as.numeric(prev_CAD_int_dataOR$asatadjbmi_lower)

prev_CAD_int_dataOR$gfatadjbmi_OR <- as.numeric(prev_CAD_int_dataOR$gfatadjbmi_OR)
prev_CAD_int_dataOR$gfatadjbmi_upper <- as.numeric(prev_CAD_int_dataOR$gfatadjbmi_upper)
prev_CAD_int_dataOR$gfatadjbmi_lower <- as.numeric(prev_CAD_int_dataOR$gfatadjbmi_lower)

prev_CAD_int_dataOR

prev_CAD_int_dataOR_print <- prev_CAD_int_dataOR[,c('subgroupName','Nfracdec',
                      'vatadjbmi_ORwCI','vatadjbmi_Pval',
                      'asatadjbmi_ORwCI','asatadjbmi_Pval',
                      'gfatadjbmi_ORwCI','gfatadjbmi_Pval')]

# write.csv(prev_CAD_int_dataOR_print,
#           './2021.03.11_paperFigs/paper1_MLandEpi/revisions/prev_cad_subgroups.csv',
#           row.names=FALSE)


## Revision: test for heterogeneity across ethnicity_subgroups

In [ ]:
library(metafor)

In [ ]:
metaSetup <- function(mod,subgroupName){
    sumtempMod <- summary(mod)
    tempCoef <- sumtempMod$coefficients
    
    temprowNames <- row.names(tempCoef)
    bmiCalcIndex <- which(temprowNames=='bmiCalcScaled')
    vatadjbmiIndex <- which(temprowNames=='vatadjbmiScaled')
    asatadjbmiIndex <- which(temprowNames=='asatadjbmiScaled')
    gfatadjbmiIndex <- which(temprowNames=='gfatadjbmiScaled')
    
    outDat <- data.table('subgroupName'=subgroupName,
                         'bmiCalc_logOR'=tempCoef[bmiCalcIndex,1],'bmiCalc_SE'=tempCoef[bmiCalcIndex,2],
                        'vatadjbmi_logOR'=tempCoef[vatadjbmiIndex,1], 'vatadjbmi_SE'=tempCoef[vatadjbmiIndex,2],
                        'asatadjbmi_logOR'=tempCoef[asatadjbmiIndex,1],'asatadjbmi_SE'=tempCoef[asatadjbmiIndex,2],
                        'gfatadjbmi_logOR'=tempCoef[gfatadjbmiIndex,1], 'gfatadjbmi_SE'=tempCoef[gfatadjbmiIndex,2])
    
    return(outDat)
}

In [ ]:
prev_T2D_meta <- rbind(metaSetup(prev_T2D_white,'White'),
                      metaSetup(prev_T2D_black,'Black'),
                      metaSetup(prev_T2D_eastAsian,'East Asian'),
                      metaSetup(prev_T2D_southAsian,'South Asian'),
                      metaSetup(prev_T2D_other,'Other'))

prev_T2D_meta

rma(vatadjbmi_logOR,vatadjbmi_SE,data=prev_T2D_meta,method='FE')
rma(asatadjbmi_logOR,asatadjbmi_SE,data=prev_T2D_meta,method='FE')
rma(gfatadjbmi_logOR,gfatadjbmi_SE,data=prev_T2D_meta,method='FE')

In [ ]:
prev_CAD_int_meta <- rbind(metaSetup(prev_CAD_int_white,'White'),
                      metaSetup(prev_CAD_int_black,'Black'),
                      metaSetup(prev_CAD_int_eastAsian,'East Asian'),
                      metaSetup(prev_CAD_int_southAsian,'South Asian'),
                      metaSetup(prev_CAD_int_other,'Other'))

prev_CAD_int_meta

rma(vatadjbmi_logOR,vatadjbmi_SE,data=prev_CAD_int_meta,method='FE')
rma(asatadjbmi_logOR,asatadjbmi_SE,data=prev_CAD_int_meta,method='FE')
rma(gfatadjbmi_logOR,gfatadjbmi_SE,data=prev_CAD_int_meta,method='FE')

In [ ]:
options(repr.plot.width=12,repr.plot.height=8)

maintext_prev_T2D_forestplot <- 

structure(list(
    mean=c(NA,NA,prev_T2D_dataOR[1]$vatadjbmi_OR,prev_T2D_dataOR[1]$asatadjbmi_OR,prev_T2D_dataOR[1]$gfatadjbmi_OR,
           NA,prev_T2D_dataOR[5]$vatadjbmi_OR,prev_T2D_dataOR[5]$asatadjbmi_OR,prev_T2D_dataOR[5]$gfatadjbmi_OR,
           NA,prev_T2D_dataOR[6]$vatadjbmi_OR,prev_T2D_dataOR[6]$asatadjbmi_OR,prev_T2D_dataOR[6]$gfatadjbmi_OR),
    lower=c(NA,NA,prev_T2D_dataOR[1]$vatadjbmi_lower,prev_T2D_dataOR[1]$asatadjbmi_lower,prev_T2D_dataOR[1]$gfatadjbmi_lower,
           NA,prev_T2D_dataOR[5]$vatadjbmi_lower,prev_T2D_dataOR[5]$asatadjbmi_lower,prev_T2D_dataOR[5]$gfatadjbmi_lower,
           NA,prev_T2D_dataOR[6]$vatadjbmi_lower,prev_T2D_dataOR[6]$asatadjbmi_lower,prev_T2D_dataOR[6]$gfatadjbmi_lower),
    upper=c(NA,NA,prev_T2D_dataOR[1]$vatadjbmi_upper,prev_T2D_dataOR[1]$asatadjbmi_upper,prev_T2D_dataOR[1]$gfatadjbmi_upper,
           NA,prev_T2D_dataOR[5]$vatadjbmi_upper,prev_T2D_dataOR[5]$asatadjbmi_upper,prev_T2D_dataOR[5]$gfatadjbmi_upper,
           NA,prev_T2D_dataOR[6]$vatadjbmi_upper,prev_T2D_dataOR[6]$asatadjbmi_upper,prev_T2D_dataOR[6]$gfatadjbmi_upper)),
    .Names=c('mean','lower','upper'),
    row.names=c(NA,-13),
    class='data.frame')

tabletext <- cbind(
c("Subgroup",NA,'All',NA,NA,NA,'Men',NA,NA,NA,'Women',NA,NA),
c("No. Affected/ \nTotal No. (%)",NA,prev_T2D_dataOR[1]$Nfracdec,NA,NA,NA,
           prev_T2D_dataOR[5]$Nfracdec,NA,NA,NA,
          prev_T2D_dataOR[6]$Nfracdec,NA,NA),
c("BMI-adjusted \nFat Depot",NA,c(rep(c('VATadjBMI','ASATadjBMI','GFATadjBMI',NA),2),c('VATadjBMI','ASATadjBMI','GFATadjBMI'))),
c("OR (95% CI)",NA,prev_T2D_dataOR[1]$vatadjbmi_ORwCI,prev_T2D_dataOR[1]$asatadjbmi_ORwCI,prev_T2D_dataOR[1]$gfatadjbmi_ORwCI,
           NA,prev_T2D_dataOR[5]$vatadjbmi_ORwCI,prev_T2D_dataOR[5]$asatadjbmi_ORwCI,prev_T2D_dataOR[5]$gfatadjbmi_ORwCI,
           NA,prev_T2D_dataOR[6]$vatadjbmi_ORwCI,prev_T2D_dataOR[6]$asatadjbmi_ORwCI,prev_T2D_dataOR[6]$gfatadjbmi_ORwCI),
c("P value",NA,prev_T2D_dataOR[1]$vatadjbmi_Pval,prev_T2D_dataOR[1]$asatadjbmi_Pval,prev_T2D_dataOR[1]$gfatadjbmi_Pval,
           NA,prev_T2D_dataOR[5]$vatadjbmi_Pval,prev_T2D_dataOR[5]$asatadjbmi_Pval,prev_T2D_dataOR[5]$gfatadjbmi_Pval,
           NA,prev_T2D_dataOR[6]$vatadjbmi_Pval,prev_T2D_dataOR[6]$asatadjbmi_Pval,prev_T2D_dataOR[6]$gfatadjbmi_Pval))

pdf(height=8,width=14,file='/medpop/esp2/sagrawal/fat_distribution/2021.03.11_paperFigs/paper1_MLandEpi/revisions/forest_T2D_maintext.pdf',
   onefile=FALSE)

forestplot(graph.pos=4,tabletext,
          maintext_prev_T2D_forestplot,
          is.summary=c(TRUE,rep(FALSE,12)),
          clip=c(0.5,1.7),
           xticks=c(0.6,1.0,1.7),
           boxsize=0.3,
          xlog=TRUE,
          col=fpColors(box=c('#BD3628'),
                      line='#BD3628'),
          lineheight='lines',
          graphwidth=unit(8,'cm'),
           xlab='Odds Ratio',
          txt_gp=fpTxtGp(ticks=gpar(fontsize=24),
                        label=gpar(fontsize=13.6),
                        xlab=gpar(fontsize=24)))

dev.off()

In [ ]:
options(repr.plot.width=12,repr.plot.height=8)

maintext_prev_CAD_int_forestplot <- 

structure(list(
    mean=c(NA,NA,prev_CAD_int_dataOR[1]$vatadjbmi_OR,prev_CAD_int_dataOR[1]$asatadjbmi_OR,prev_CAD_int_dataOR[1]$gfatadjbmi_OR,
           NA,prev_CAD_int_dataOR[5]$vatadjbmi_OR,prev_CAD_int_dataOR[5]$asatadjbmi_OR,prev_CAD_int_dataOR[5]$gfatadjbmi_OR,
           NA,prev_CAD_int_dataOR[6]$vatadjbmi_OR,prev_CAD_int_dataOR[6]$asatadjbmi_OR,prev_CAD_int_dataOR[6]$gfatadjbmi_OR),
    lower=c(NA,NA,prev_CAD_int_dataOR[1]$vatadjbmi_lower,prev_CAD_int_dataOR[1]$asatadjbmi_lower,prev_CAD_int_dataOR[1]$gfatadjbmi_lower,
           NA,prev_CAD_int_dataOR[5]$vatadjbmi_lower,prev_CAD_int_dataOR[5]$asatadjbmi_lower,prev_CAD_int_dataOR[5]$gfatadjbmi_lower,
           NA,prev_CAD_int_dataOR[6]$vatadjbmi_lower,prev_CAD_int_dataOR[6]$asatadjbmi_lower,prev_CAD_int_dataOR[6]$gfatadjbmi_lower),
    upper=c(NA,NA,prev_CAD_int_dataOR[1]$vatadjbmi_upper,prev_CAD_int_dataOR[1]$asatadjbmi_upper,prev_CAD_int_dataOR[1]$gfatadjbmi_upper,
           NA,prev_CAD_int_dataOR[5]$vatadjbmi_upper,prev_CAD_int_dataOR[5]$asatadjbmi_upper,prev_CAD_int_dataOR[5]$gfatadjbmi_upper,
           NA,prev_CAD_int_dataOR[6]$vatadjbmi_upper,prev_CAD_int_dataOR[6]$asatadjbmi_upper,prev_CAD_int_dataOR[6]$gfatadjbmi_upper)),
    .Names=c('mean','lower','upper'),
    row.names=c(NA,-13),
    class='data.frame')

tabletext <- cbind(
c("Subgroup",NA,'All',NA,NA,NA,'Men',NA,NA,NA,'Women',NA,NA),
c("No. Affected/ \nTotal No. (%)",NA,prev_CAD_int_dataOR[1]$Nfracdec,NA,NA,NA,
           prev_CAD_int_dataOR[5]$Nfracdec,NA,NA,NA,
          prev_CAD_int_dataOR[6]$Nfracdec,NA,NA),
c("BMI-adjusted \nFat Depot",NA,c(rep(c('VATadjBMI','ASATadjBMI','GFATadjBMI',NA),2),c('VATadjBMI','ASATadjBMI','GFATadjBMI'))),
c("OR (95% CI)",NA,prev_CAD_int_dataOR[1]$vatadjbmi_ORwCI,prev_CAD_int_dataOR[1]$asatadjbmi_ORwCI,prev_CAD_int_dataOR[1]$gfatadjbmi_ORwCI,
           NA,prev_CAD_int_dataOR[5]$vatadjbmi_ORwCI,prev_CAD_int_dataOR[5]$asatadjbmi_ORwCI,prev_CAD_int_dataOR[5]$gfatadjbmi_ORwCI,
           NA,prev_CAD_int_dataOR[6]$vatadjbmi_ORwCI,prev_CAD_int_dataOR[6]$asatadjbmi_ORwCI,prev_CAD_int_dataOR[6]$gfatadjbmi_ORwCI),
c("P value",NA,prev_CAD_int_dataOR[1]$vatadjbmi_Pval,prev_CAD_int_dataOR[1]$asatadjbmi_Pval,prev_CAD_int_dataOR[1]$gfatadjbmi_Pval,
           NA,prev_CAD_int_dataOR[5]$vatadjbmi_Pval,prev_CAD_int_dataOR[5]$asatadjbmi_Pval,prev_CAD_int_dataOR[5]$gfatadjbmi_Pval,
           NA,prev_CAD_int_dataOR[6]$vatadjbmi_Pval,prev_CAD_int_dataOR[6]$asatadjbmi_Pval,prev_CAD_int_dataOR[6]$gfatadjbmi_Pval))

pdf(height=8,width=14,file='/medpop/esp2/sagrawal/fat_distribution/2021.03.11_paperFigs/paper1_MLandEpi/revisions/forest_CAD_int_maintext.pdf',
   onefile=FALSE)

forestplot(graph.pos=4,tabletext,
          maintext_prev_CAD_int_forestplot,
          is.summary=c(TRUE,rep(FALSE,12)),
          clip=c(0.5,1.7),
           xticks=c(0.6,1.0,1.7),
           boxsize=0.3,
          xlog=TRUE,
          col=fpColors(box=c('#BD3628'),
                      line='#BD3628'),
          lineheight='lines',
          graphwidth=unit(8,'cm'),
           xlab='Odds Ratio',
          txt_gp=fpTxtGp(ticks=gpar(fontsize=24),
                        label=gpar(fontsize=13.6),
                        xlab=gpar(fontsize=24)))

dev.off()

In [ ]:
options(repr.plot.width=12,repr.plot.height=8)

vatadjbmi_prev_T2D_forestplot <- 

structure(list(
    mean=c(NA,NA,prev_T2D_dataOR[1]$vatadjbmi_OR,
           NA,prev_T2D_dataOR[2:4]$vatadjbmi_OR,
           NA,prev_T2D_dataOR[5:6]$vatadjbmi_OR,
           NA,prev_T2D_dataOR[7:9]$vatadjbmi_OR),
    lower=c(NA,NA,prev_T2D_dataOR[1]$vatadjbmi_lower,
           NA,prev_T2D_dataOR[2:4]$vatadjbmi_lower,
           NA,prev_T2D_dataOR[5:6]$vatadjbmi_lower,
           NA,prev_T2D_dataOR[7:9]$vatadjbmi_lower),
    upper=c(NA,NA,prev_T2D_dataOR[1]$vatadjbmi_upper,
           NA,prev_T2D_dataOR[2:4]$vatadjbmi_upper,
           NA,prev_T2D_dataOR[5:6]$vatadjbmi_upper,
           NA,prev_T2D_dataOR[7:9]$vatadjbmi_upper)),
    .Names=c('mean','lower','upper'),
    row.names=c(NA,-14),
    class='data.frame')

tabletext <- cbind(
c("Subgroup",NA,'All',
           "Age Strata","< 60", "60 <= Age < 70", ">=70",
           "Sex",'Men','Women',
           "BMI Strata","< 25", "25 <= BMI < 30", ">=30"),
c("No. Affected/ \nTotal No. (%)",NA,prev_T2D_dataOR[1]$Nfracdec,
           NA,prev_T2D_dataOR[2:4]$Nfracdec,
           NA,prev_T2D_dataOR[5:6]$Nfracdec,
           NA,prev_T2D_dataOR[7:9]$Nfracdec),
c("VATadjBMI \nOR (95% CI)",NA,prev_T2D_dataOR[1]$vatadjbmi_ORwCI,
           NA,prev_T2D_dataOR[2:4]$vatadjbmi_ORwCI,
           NA,prev_T2D_dataOR[5:6]$vatadjbmi_ORwCI,
           NA,prev_T2D_dataOR[7:9]$vatadjbmi_ORwCI),
c("P value",NA,prev_T2D_dataOR[1]$vatadjbmi_Pval,
           NA,prev_T2D_dataOR[2:4]$vatadjbmi_Pval,
           NA,prev_T2D_dataOR[5:6]$vatadjbmi_Pval,
           NA,prev_T2D_dataOR[7:9]$vatadjbmi_Pval))

pdf(height=8,width=14,file='/medpop/esp2/sagrawal/fat_distribution/2021.03.11_paperFigs/paper1_MLandEpi/revisions/forest_T2D_subgroups_vatadjbmi.pdf',
   onefile=FALSE)

forestplot(graph.pos=3,tabletext,
          vatadjbmi_prev_T2D_forestplot,
          is.summary=c(TRUE,FALSE,FALSE,TRUE,rep(FALSE,3),TRUE,rep(FALSE,2),TRUE,rep(FALSE,3)),
          clip=c(0.5,2.0),
           xticks=c(0.5,1.0,2.0),
           boxsize=0.3,
          xlog=TRUE,
          col=fpColors(box=c('#e53377'),
                      line='#e53377'),
          lineheight='lines',
          graphwidth=unit(8,'cm'),
           xlab='Odds Ratio',
          txt_gp=fpTxtGp(ticks=gpar(fontsize=24),
                        label=gpar(fontsize=13.6),
                        xlab=gpar(fontsize=24)))

dev.off()

In [ ]:
options(repr.plot.width=12,repr.plot.height=8)

asatadjbmi_prev_T2D_forestplot <- 

structure(list(
    mean=c(NA,NA,prev_T2D_dataOR[1]$asatadjbmi_OR,
           NA,prev_T2D_dataOR[2:4]$asatadjbmi_OR,
           NA,prev_T2D_dataOR[5:6]$asatadjbmi_OR,
           NA,prev_T2D_dataOR[7:9]$asatadjbmi_OR),
    lower=c(NA,NA,prev_T2D_dataOR[1]$asatadjbmi_lower,
           NA,prev_T2D_dataOR[2:4]$asatadjbmi_lower,
           NA,prev_T2D_dataOR[5:6]$asatadjbmi_lower,
           NA,prev_T2D_dataOR[7:9]$asatadjbmi_lower),
    upper=c(NA,NA,prev_T2D_dataOR[1]$asatadjbmi_upper,
           NA,prev_T2D_dataOR[2:4]$asatadjbmi_upper,
           NA,prev_T2D_dataOR[5:6]$asatadjbmi_upper,
           NA,prev_T2D_dataOR[7:9]$asatadjbmi_upper)),
    .Names=c('mean','lower','upper'),
    row.names=c(NA,-14),
    class='data.frame')

tabletext <- cbind(
c("Subgroup",NA,'All',
           "Age Strata","< 60", "60 <= Age < 70", ">=70",
           "Sex",'Men','Women',
           "BMI Strata","< 25", "25 <= BMI < 30", ">=30"),
c("No. Affected/ \nTotal No. (%)",NA,prev_T2D_dataOR[1]$Nfracdec,
           NA,prev_T2D_dataOR[2:4]$Nfracdec,
           NA,prev_T2D_dataOR[5:6]$Nfracdec,
           NA,prev_T2D_dataOR[7:9]$Nfracdec),
c("ASATadjBMI \nOR (95% CI)",NA,prev_T2D_dataOR[1]$asatadjbmi_ORwCI,
           NA,prev_T2D_dataOR[2:4]$asatadjbmi_ORwCI,
           NA,prev_T2D_dataOR[5:6]$asatadjbmi_ORwCI,
           NA,prev_T2D_dataOR[7:9]$asatadjbmi_ORwCI),
c("P value",NA,prev_T2D_dataOR[1]$asatadjbmi_Pval,
           NA,prev_T2D_dataOR[2:4]$asatadjbmi_Pval,
           NA,prev_T2D_dataOR[5:6]$asatadjbmi_Pval,
           NA,prev_T2D_dataOR[7:9]$asatadjbmi_Pval))

pdf(height=8,width=14,file='/medpop/esp2/sagrawal/fat_distribution/2021.03.11_paperFigs/paper1_MLandEpi/revisions/forest_T2D_subgroups_asatadjbmi.pdf',
   onefile=FALSE)

forestplot(graph.pos=3,tabletext,
          asatadjbmi_prev_T2D_forestplot,
          is.summary=c(TRUE,FALSE,FALSE,TRUE,rep(FALSE,3),TRUE,rep(FALSE,2),TRUE,rep(FALSE,3)),
          clip=c(0.5,2.0),
           xticks=c(0.5,1.0,2.0),
           boxsize=0.3,
          xlog=TRUE,
          col=fpColors(box=c('#ecb31f'),
                      line='#ecb31f'),
          lineheight='lines',
          graphwidth=unit(8,'cm'),
           xlab='Odds Ratio',
          txt_gp=fpTxtGp(ticks=gpar(fontsize=24),
                        label=gpar(fontsize=13.6),
                        xlab=gpar(fontsize=24)))

dev.off()

In [ ]:
options(repr.plot.width=12,repr.plot.height=8)

gfatadjbmi_prev_T2D_forestplot <- 

structure(list(
    mean=c(NA,NA,prev_T2D_dataOR[1]$gfatadjbmi_OR,
           NA,prev_T2D_dataOR[2:4]$gfatadjbmi_OR,
           NA,prev_T2D_dataOR[5:6]$gfatadjbmi_OR,
           NA,prev_T2D_dataOR[7:9]$gfatadjbmi_OR),
    lower=c(NA,NA,prev_T2D_dataOR[1]$gfatadjbmi_lower,
           NA,prev_T2D_dataOR[2:4]$gfatadjbmi_lower,
           NA,prev_T2D_dataOR[5:6]$gfatadjbmi_lower,
           NA,prev_T2D_dataOR[7:9]$gfatadjbmi_lower),
    upper=c(NA,NA,prev_T2D_dataOR[1]$gfatadjbmi_upper,
           NA,prev_T2D_dataOR[2:4]$gfatadjbmi_upper,
           NA,prev_T2D_dataOR[5:6]$gfatadjbmi_upper,
           NA,prev_T2D_dataOR[7:9]$gfatadjbmi_upper)),
    .Names=c('mean','lower','upper'),
    row.names=c(NA,-14),
    class='data.frame')

tabletext <- cbind(
c("Subgroup",NA,'All',
           "Age Strata","< 60", "60 <= Age < 70", ">=70",
           "Sex",'Men','Women',
           "BMI Strata","< 25", "25 <= BMI < 30", ">=30"),
c("No. Affected/ \nTotal No. (%)",NA,prev_T2D_dataOR[1]$Nfracdec,
           NA,prev_T2D_dataOR[2:4]$Nfracdec,
           NA,prev_T2D_dataOR[5:6]$Nfracdec,
           NA,prev_T2D_dataOR[7:9]$Nfracdec),
c("GFATadjBMI \nOR (95% CI)",NA,prev_T2D_dataOR[1]$gfatadjbmi_ORwCI,
           NA,prev_T2D_dataOR[2:4]$gfatadjbmi_ORwCI,
           NA,prev_T2D_dataOR[5:6]$gfatadjbmi_ORwCI,
           NA,prev_T2D_dataOR[7:9]$gfatadjbmi_ORwCI),
c("P value",NA,prev_T2D_dataOR[1]$gfatadjbmi_Pval,
           NA,prev_T2D_dataOR[2:4]$gfatadjbmi_Pval,
           NA,prev_T2D_dataOR[5:6]$gfatadjbmi_Pval,
           NA,prev_T2D_dataOR[7:9]$gfatadjbmi_Pval))

pdf(height=8,width=14,file='/medpop/esp2/sagrawal/fat_distribution/2021.03.11_paperFigs/paper1_MLandEpi/revisions/forest_T2D_subgroups_gfatadjbmi.pdf',
   onefile=FALSE)

forestplot(graph.pos=3,tabletext,
          gfatadjbmi_prev_T2D_forestplot,
          is.summary=c(TRUE,FALSE,FALSE,TRUE,rep(FALSE,3),TRUE,rep(FALSE,2),TRUE,rep(FALSE,3)),
          clip=c(0.5,2.0),
           xticks=c(0.5,1.0,2.0),
           boxsize=0.3,
          xlog=TRUE,
          col=fpColors(box=c('#27b2af'),
                      line='#27b2af'),
          lineheight='lines',
          graphwidth=unit(8,'cm'),
           xlab='Odds Ratio',
          txt_gp=fpTxtGp(ticks=gpar(fontsize=24),
                        label=gpar(fontsize=13.6),
                        xlab=gpar(fontsize=24)))

dev.off()

In [ ]:
options(repr.plot.width=12,repr.plot.height=8)

vatadjbmi_prev_CAD_int_forestplot <- 

structure(list(
    mean=c(NA,NA,prev_CAD_int_dataOR[1]$vatadjbmi_OR,
           NA,prev_CAD_int_dataOR[2:4]$vatadjbmi_OR,
           NA,prev_CAD_int_dataOR[5:6]$vatadjbmi_OR,
           NA,prev_CAD_int_dataOR[7:9]$vatadjbmi_OR),
    lower=c(NA,NA,prev_CAD_int_dataOR[1]$vatadjbmi_lower,
           NA,prev_CAD_int_dataOR[2:4]$vatadjbmi_lower,
           NA,prev_CAD_int_dataOR[5:6]$vatadjbmi_lower,
           NA,prev_CAD_int_dataOR[7:9]$vatadjbmi_lower),
    upper=c(NA,NA,prev_CAD_int_dataOR[1]$vatadjbmi_upper,
           NA,prev_CAD_int_dataOR[2:4]$vatadjbmi_upper,
           NA,prev_CAD_int_dataOR[5:6]$vatadjbmi_upper,
           NA,prev_CAD_int_dataOR[7:9]$vatadjbmi_upper)),
    .Names=c('mean','lower','upper'),
    row.names=c(NA,-14),
    class='data.frame')

tabletext <- cbind(
c("Subgroup",NA,'All',
           "Age Strata","< 60", "60 <= Age < 70", ">=70",
           "Sex",'Men','Women',
           "BMI Strata","< 25", "25 <= BMI < 30", ">=30"),
c("No. Affected/ \nTotal No. (%)",NA,prev_CAD_int_dataOR[1]$Nfracdec,
           NA,prev_CAD_int_dataOR[2:4]$Nfracdec,
           NA,prev_CAD_int_dataOR[5:6]$Nfracdec,
           NA,prev_CAD_int_dataOR[7:9]$Nfracdec),
c("VATadjBMI \nOR (95% CI)",NA,prev_CAD_int_dataOR[1]$vatadjbmi_ORwCI,
           NA,prev_CAD_int_dataOR[2:4]$vatadjbmi_ORwCI,
           NA,prev_CAD_int_dataOR[5:6]$vatadjbmi_ORwCI,
           NA,prev_CAD_int_dataOR[7:9]$vatadjbmi_ORwCI),
c("P value",NA,prev_CAD_int_dataOR[1]$vatadjbmi_Pval,
           NA,prev_CAD_int_dataOR[2:4]$vatadjbmi_Pval,
           NA,prev_CAD_int_dataOR[5:6]$vatadjbmi_Pval,
           NA,prev_CAD_int_dataOR[7:9]$vatadjbmi_Pval))

pdf(height=8,width=14,file='/medpop/esp2/sagrawal/fat_distribution/2021.03.11_paperFigs/paper1_MLandEpi/revisions/forest_CAD_int_subgroups_vatadjbmi.pdf',
   onefile=FALSE)

forestplot(graph.pos=3,tabletext,
          vatadjbmi_prev_CAD_int_forestplot,
          is.summary=c(TRUE,FALSE,FALSE,TRUE,rep(FALSE,3),TRUE,rep(FALSE,2),TRUE,rep(FALSE,3)),
          clip=c(0.5,2.0),
           xticks=c(0.5,1.0,2.0),
           boxsize=0.3,
          xlog=TRUE,
          col=fpColors(box=c('#e53377'),
                      line='#e53377'),
          lineheight='lines',
          graphwidth=unit(8,'cm'),
           xlab='Odds Ratio',
          txt_gp=fpTxtGp(ticks=gpar(fontsize=24),
                        label=gpar(fontsize=13.6),
                        xlab=gpar(fontsize=24)))

dev.off()

In [ ]:
options(repr.plot.width=12,repr.plot.height=8)

asatadjbmi_prev_CAD_int_forestplot <- 

structure(list(
    mean=c(NA,NA,prev_CAD_int_dataOR[1]$asatadjbmi_OR,
           NA,prev_CAD_int_dataOR[2:4]$asatadjbmi_OR,
           NA,prev_CAD_int_dataOR[5:6]$asatadjbmi_OR,
           NA,prev_CAD_int_dataOR[7:9]$asatadjbmi_OR),
    lower=c(NA,NA,prev_CAD_int_dataOR[1]$asatadjbmi_lower,
           NA,prev_CAD_int_dataOR[2:4]$asatadjbmi_lower,
           NA,prev_CAD_int_dataOR[5:6]$asatadjbmi_lower,
           NA,prev_CAD_int_dataOR[7:9]$asatadjbmi_lower),
    upper=c(NA,NA,prev_CAD_int_dataOR[1]$asatadjbmi_upper,
           NA,prev_CAD_int_dataOR[2:4]$asatadjbmi_upper,
           NA,prev_CAD_int_dataOR[5:6]$asatadjbmi_upper,
           NA,prev_CAD_int_dataOR[7:9]$asatadjbmi_upper)),
    .Names=c('mean','lower','upper'),
    row.names=c(NA,-14),
    class='data.frame')

tabletext <- cbind(
c("Subgroup",NA,'All',
           "Age Strata","< 60", "60 <= Age < 70", ">=70",
           "Sex",'Men','Women',
           "BMI Strata","< 25", "25 <= BMI < 30", ">=30"),
c("No. Affected/ \nTotal No. (%)",NA,prev_CAD_int_dataOR[1]$Nfracdec,
           NA,prev_CAD_int_dataOR[2:4]$Nfracdec,
           NA,prev_CAD_int_dataOR[5:6]$Nfracdec,
           NA,prev_CAD_int_dataOR[7:9]$Nfracdec),
c("ASATadjBMI \nOR (95% CI)",NA,prev_CAD_int_dataOR[1]$asatadjbmi_ORwCI,
           NA,prev_CAD_int_dataOR[2:4]$asatadjbmi_ORwCI,
           NA,prev_CAD_int_dataOR[5:6]$asatadjbmi_ORwCI,
           NA,prev_CAD_int_dataOR[7:9]$asatadjbmi_ORwCI),
c("P value",NA,prev_CAD_int_dataOR[1]$asatadjbmi_Pval,
           NA,prev_CAD_int_dataOR[2:4]$asatadjbmi_Pval,
           NA,prev_CAD_int_dataOR[5:6]$asatadjbmi_Pval,
           NA,prev_CAD_int_dataOR[7:9]$asatadjbmi_Pval))

pdf(height=8,width=14,file='/medpop/esp2/sagrawal/fat_distribution/2021.03.11_paperFigs/paper1_MLandEpi/revisions/forest_CAD_int_subgroups_asatadjbmi.pdf',
   onefile=FALSE)

forestplot(graph.pos=3,tabletext,
          asatadjbmi_prev_CAD_int_forestplot,
          is.summary=c(TRUE,FALSE,FALSE,TRUE,rep(FALSE,3),TRUE,rep(FALSE,2),TRUE,rep(FALSE,3)),
          clip=c(0.5,2.0),
           xticks=c(0.5,1.0,2.0),
           boxsize=0.3,
          xlog=TRUE,
          col=fpColors(box=c('#ecb31f'),
                      line='#ecb31f'),
          lineheight='lines',
          graphwidth=unit(8,'cm'),
           xlab='Odds Ratio',
          txt_gp=fpTxtGp(ticks=gpar(fontsize=24),
                        label=gpar(fontsize=13.6),
                        xlab=gpar(fontsize=24)))

dev.off()

In [ ]:
options(repr.plot.width=12,repr.plot.height=8)

gfatadjbmi_prev_CAD_int_forestplot <- 

structure(list(
    mean=c(NA,NA,prev_CAD_int_dataOR[1]$gfatadjbmi_OR,
           NA,prev_CAD_int_dataOR[2:4]$gfatadjbmi_OR,
           NA,prev_CAD_int_dataOR[5:6]$gfatadjbmi_OR,
           NA,prev_CAD_int_dataOR[7:9]$gfatadjbmi_OR),
    lower=c(NA,NA,prev_CAD_int_dataOR[1]$gfatadjbmi_lower,
           NA,prev_CAD_int_dataOR[2:4]$gfatadjbmi_lower,
           NA,prev_CAD_int_dataOR[5:6]$gfatadjbmi_lower,
           NA,prev_CAD_int_dataOR[7:9]$gfatadjbmi_lower),
    upper=c(NA,NA,prev_CAD_int_dataOR[1]$gfatadjbmi_upper,
           NA,prev_CAD_int_dataOR[2:4]$gfatadjbmi_upper,
           NA,prev_CAD_int_dataOR[5:6]$gfatadjbmi_upper,
           NA,prev_CAD_int_dataOR[7:9]$gfatadjbmi_upper)),
    .Names=c('mean','lower','upper'),
    row.names=c(NA,-14),
    class='data.frame')

tabletext <- cbind(
c("Subgroup",NA,'All',
           "Age Strata","< 60", "60 <= Age < 70", ">=70",
           "Sex",'Men','Women',
           "BMI Strata","< 25", "25 <= BMI < 30", ">=30"),
c("No. Affected/ \nTotal No. (%)",NA,prev_CAD_int_dataOR[1]$Nfracdec,
           NA,prev_CAD_int_dataOR[2:4]$Nfracdec,
           NA,prev_CAD_int_dataOR[5:6]$Nfracdec,
           NA,prev_CAD_int_dataOR[7:9]$Nfracdec),
c("GFATadjBMI \nOR (95% CI)",NA,prev_CAD_int_dataOR[1]$gfatadjbmi_ORwCI,
           NA,prev_CAD_int_dataOR[2:4]$gfatadjbmi_ORwCI,
           NA,prev_CAD_int_dataOR[5:6]$gfatadjbmi_ORwCI,
           NA,prev_CAD_int_dataOR[7:9]$gfatadjbmi_ORwCI),
c("P value",NA,prev_CAD_int_dataOR[1]$gfatadjbmi_Pval,
           NA,prev_CAD_int_dataOR[2:4]$gfatadjbmi_Pval,
           NA,prev_CAD_int_dataOR[5:6]$gfatadjbmi_Pval,
           NA,prev_CAD_int_dataOR[7:9]$gfatadjbmi_Pval))

pdf(height=8,width=14,file='/medpop/esp2/sagrawal/fat_distribution/2021.03.11_paperFigs/paper1_MLandEpi/revisions/forest_CAD_int_subgroups_gfatadjbmi.pdf',
   onefile=FALSE)

forestplot(graph.pos=3,tabletext,
          gfatadjbmi_prev_CAD_int_forestplot,
          is.summary=c(TRUE,FALSE,FALSE,TRUE,rep(FALSE,3),TRUE,rep(FALSE,2),TRUE,rep(FALSE,3)),
          clip=c(0.5,2.0),
           xticks=c(0.5,1.0,2.0),
           boxsize=0.3,
          xlog=TRUE,
          col=fpColors(box=c('#27b2af'),
                      line='#27b2af'),
          lineheight='lines',
          graphwidth=unit(8,'cm'),
           xlab='Odds Ratio',
          txt_gp=fpTxtGp(ticks=gpar(fontsize=24),
                        label=gpar(fontsize=13.6),
                        xlab=gpar(fontsize=24)))

dev.off()

# Standardized prevalence with sex-specific models

In [ ]:
prev_T2D_plotter_Male <- glm(prev_T2D~age_instance2+
                      bmiCalcScaled*(vatadjbmiQuintilesStrat+asatadjbmiQuintilesStrat+gfatadjbmiQuintilesStrat)+
                      mriNum_141243+mriNum_141631,
                      data=trainDat[sex=='Male'],family='binomial')

summary(prev_T2D_plotter_Male)

prev_T2D_plotter_Female <- glm(prev_T2D~age_instance2+
                      bmiCalcScaled*(vatadjbmiQuintilesStrat+asatadjbmiQuintilesStrat+gfatadjbmiQuintilesStrat)+
                      mriNum_141243+mriNum_141631,
                      data=trainDat[sex=='Female'],family='binomial')

summary(prev_T2D_plotter_Female)


prev_CAD_plotter_Male <- glm(prev_CAD_int~age_instance2+
                      bmiCalcScaled*(vatadjbmiQuintilesStrat+asatadjbmiQuintilesStrat+gfatadjbmiQuintilesStrat)+
                      mriNum_141243+mriNum_141631,
                      data=trainDat[sex=='Male'],family='binomial')

summary(prev_CAD_plotter_Male)

prev_CAD_plotter_Female <- glm(prev_CAD_int~age_instance2+
                      bmiCalcScaled*(vatadjbmiQuintilesStrat+asatadjbmiQuintilesStrat+gfatadjbmiQuintilesStrat)+
                      mriNum_141243+mriNum_141631,
                      data=trainDat[sex=='Female'],family='binomial')

summary(prev_CAD_plotter_Female)

In [ ]:
sliderDatWrapper <- function(bmiGroupVar,sexVar,dat){
    dat2 <- copy(dat)
    
    ageTemp <- median(dat2[sex==sexVar&bmiGroup==bmiGroupVar]$age_instance2)
    bmiTemp <- median(dat2[sex==sexVar&bmiGroup==bmiGroupVar]$bmiCalcScaled)
    mriNum1 <- mean(dat2[sex==sexVar&bmiGroup==bmiGroupVar]$mriNum_141243)
    mriNum2 <- mean(dat2[sex==sexVar&bmiGroup==bmiGroupVar]$mriNum_141631)
    
    predTable <- data.table('age_instance2'=rep(ageTemp,9),'sex'=rep(sexVar,9),
                                'bmiCalcScaled'=rep(bmiTemp,9),'mriNum_141243'=rep(mriNum1,9),
                                'mriNum_141631'=rep(mriNum2,9),'Percentile'=rep(c('Q1','Q2_Q4','Q5'),3),
                             'vatadjbmiQuintilesStrat'=c('Q1','Q2_Q4','Q5',rep('Q2_Q4',6)),
                             'asatadjbmiQuintilesStrat'=c(rep('Q2_Q4',3),'Q1','Q2_Q4','Q5',rep('Q2_Q4',3)),
                             'gfatadjbmiQuintilesStrat'=c(rep('Q2_Q4',6),'Q1','Q2_Q4','Q5'),
                                'indepVar'=c(rep('VATadjBMI',3),rep('ASATadjBMI',3),rep('GFATadjBMI',3)))
    
    predTable$indepVar <- factor(predTable$indepVar,levels=c('VATadjBMI','ASATadjBMI','GFATadjBMI'))
    
    return(predTable)
}

In [ ]:
annotatesize <- 11.5*(5/14)

sliderPlotWrapper_1 <- function(dat,model,yax=TRUE){
    dat$predLog <- predict(model,dat,se.fit=TRUE)$fit
    dat$seLog <- predict(model,dat,se.fit=TRUE)$se.fit
    dat[,predAbs:=unlogit(predLog)]
    dat[,predUpper:=unlogit(predLog+(1.96*seLog))]
    dat[,predLower:=unlogit(predLog-(1.96*seLog))]
    
    g1 <- ggplot(dat[indepVar=='VATadjBMI'],aes(x=indepVar,y=predAbs*100,fill=Percentile)) +
    geom_bar(stat='identity',position='dodge') +
    geom_errorbar(aes(ymin=predLower*100,ymax=predUpper*100),
                  width=0.25,position=position_dodge(width=0.9),color='black',alpha=0.2)+
    geom_text(aes(label=format(round(predAbs*100,1),nsmall=1)),position=position_dodge(width=0.9),vjust=-0.8,size=annotatesize)+
    theme_classic() +
    xlab('') +
    ylab('Type 2 Diabetes, %') +
    scale_y_continuous(limits=c(0,18),breaks=seq(0,16,4),expand=c(0,0)) +
#     scale_x_continuous(limits=c(0,100),breaks=seq(0,100,25)) +
    scale_fill_manual(values=c('#f4adc8','#ec709f','#e53377')) +
    theme_classic() +
    theme(axis.title.y = element_text(size=18),
          axis.text.y = element_text(size=14,color='black'),
          axis.ticks.y = element_blank(),
          axis.title.x = element_text(size=18),
          axis.text.x = element_text(size=12,color='black'),
          axis.ticks.x = element_blank(),
          legend.text.align=0,
         legend.key.size=unit(1,'cm'),
         legend.title = element_text(size=16),
         legend.text = element_text(size=16),
         legend.position='none')
    
    g2 <- ggplot(dat[indepVar=='ASATadjBMI'],aes(x=indepVar,y=predAbs*100,fill=Percentile)) +
    geom_bar(stat='identity',position='dodge') +
    geom_errorbar(aes(ymin=predLower*100,ymax=predUpper*100),
                  width=0.25,position=position_dodge(width=0.9),color='black',alpha=0.2)+
    geom_text(aes(label=format(round(predAbs*100,1),nsmall=1)),position=position_dodge(width=0.9),vjust=-0.8,size=annotatesize)+
    theme_classic() +
    xlab('') +
    ylab('') +
    scale_y_continuous(limits=c(0,18),breaks=seq(0,16,4),expand=c(0,0)) +
#     scale_x_continuous(limits=c(0,100),breaks=seq(0,100,25)) +
    scale_fill_manual(values=c('#f7e0a5','#f1c962','#ecb31f')) +
    theme_classic() +
    theme(axis.title.y = element_blank(),
          axis.text.y = element_blank(),
          axis.ticks.y = element_blank(),
          axis.title.x = element_blank(),
          axis.text.x = element_text(size=12,color='black'),
          axis.ticks.x = element_blank(),
          axis.line.y=element_blank(),
          legend.text.align=0,
         legend.key.size=unit(1,'cm'),
         legend.title = element_text(size=16),
         legend.text = element_text(size=16),
         legend.position='none')
    
    g3 <- ggplot(dat[indepVar=='GFATadjBMI'],aes(x=indepVar,y=predAbs*100,fill=Percentile)) +
    geom_bar(stat='identity',position='dodge') +
    geom_errorbar(aes(ymin=predLower*100,ymax=predUpper*100),
                  width=0.25,position=position_dodge(width=0.9),color='black',alpha=0.2)+
    geom_text(aes(label=format(round(predAbs*100,1),nsmall=1)),position=position_dodge(width=0.9),vjust=-0.8,size=annotatesize)+
    theme_classic() +
    xlab('') +
    ylab('') +
    scale_y_continuous(limits=c(0,18),breaks=seq(0,16,4),expand=c(0,0)) +
#     scale_x_continuous(limits=c(0,100),breaks=seq(0,100,25)) +
    scale_fill_manual(values=c('#a8e0df','#67c9c7','#27b2af')) +
    theme_classic() +
    theme(axis.title.y = element_blank(),
          axis.text.y = element_blank(),
          axis.ticks.y = element_blank(),
          axis.title.x = element_blank(),
          axis.text.x = element_text(size=12,color='black'),
          axis.ticks.x = element_blank(),
          axis.line.y=element_blank(),
          legend.text.align=0,
         legend.key.size=unit(1,'cm'),
         legend.title = element_text(size=16),
         legend.text = element_text(size=16),
         legend.position='none')
    
    if(yax==FALSE){
        g1 <- g1+ylab('')
    }
    
    g4 <- g1+g2+g3
    
    return(g4)
}

In [ ]:
sliderPlotWrapper_1_female <- function(dat,model,yax=TRUE){
    dat$predLog <- predict(model,dat,se.fit=TRUE)$fit
    dat$seLog <- predict(model,dat,se.fit=TRUE)$se.fit
    dat[,predAbs:=unlogit(predLog)]
    dat[,predUpper:=unlogit(predLog+(1.96*seLog))]
    dat[,predLower:=unlogit(predLog-(1.96*seLog))]
    
    g1 <- ggplot(dat[indepVar=='VATadjBMI'],aes(x=indepVar,y=predAbs*100,fill=Percentile)) +
    geom_bar(stat='identity',position='dodge') +
    geom_errorbar(aes(ymin=predLower*100,ymax=predUpper*100),
                  width=0.25,position=position_dodge(width=0.9),color='black',alpha=0.2)+
    geom_text(aes(label=format(round(predAbs*100,1),nsmall=1)),position=position_dodge(width=0.9),vjust=-0.8,size=annotatesize)+
    theme_classic() +
    xlab('') +
    ylab('Type 2 Diabetes, %') +
    scale_y_continuous(limits=c(0,12*1.125),breaks=seq(0,12,4),expand=c(0,0)) +
#     scale_x_continuous(limits=c(0,100),breaks=seq(0,100,25)) +
    scale_fill_manual(values=c('#f4adc8','#ec709f','#e53377')) +
    theme_classic() +
    theme(axis.title.y = element_text(size=18),
          axis.text.y = element_text(size=14,color='black'),
          axis.ticks.y = element_blank(),
          axis.title.x = element_text(size=18),
          axis.text.x = element_text(size=12,color='black'),
          axis.ticks.x = element_blank(),
          legend.text.align=0,
         legend.key.size=unit(1,'cm'),
         legend.title = element_text(size=16),
         legend.text = element_text(size=16),
         legend.position='none')
    
    g2 <- ggplot(dat[indepVar=='ASATadjBMI'],aes(x=indepVar,y=predAbs*100,fill=Percentile)) +
    geom_bar(stat='identity',position='dodge') +
    geom_errorbar(aes(ymin=predLower*100,ymax=predUpper*100),
                  width=0.25,position=position_dodge(width=0.9),color='black',alpha=0.2)+
    geom_text(aes(label=format(round(predAbs*100,1),nsmall=1)),position=position_dodge(width=0.9),vjust=-0.8,size=annotatesize)+
    theme_classic() +
    xlab('') +
    ylab('') +
    scale_y_continuous(limits=c(0,12*1.125),breaks=seq(0,12,4),expand=c(0,0)) +
#     scale_x_continuous(limits=c(0,100),breaks=seq(0,100,25)) +
    scale_fill_manual(values=c('#f7e0a5','#f1c962','#ecb31f')) +
    theme_classic() +
    theme(axis.title.y = element_blank(),
          axis.text.y = element_blank(),
          axis.ticks.y = element_blank(),
          axis.title.x = element_blank(),
          axis.text.x = element_text(size=12,color='black'),
          axis.ticks.x = element_blank(),
          axis.line.y=element_blank(),
          legend.text.align=0,
         legend.key.size=unit(1,'cm'),
         legend.title = element_text(size=16),
         legend.text = element_text(size=16),
         legend.position='none')
    
    g3 <- ggplot(dat[indepVar=='GFATadjBMI'],aes(x=indepVar,y=predAbs*100,fill=Percentile)) +
    geom_bar(stat='identity',position='dodge') +
    geom_errorbar(aes(ymin=predLower*100,ymax=predUpper*100),
                  width=0.25,position=position_dodge(width=0.9),color='black',alpha=0.2)+
    geom_text(aes(label=format(round(predAbs*100,1),nsmall=1)),position=position_dodge(width=0.9),vjust=-0.8,size=annotatesize)+
    theme_classic() +
    xlab('') +
    ylab('') +
    scale_y_continuous(limits=c(0,12*1.125),breaks=seq(0,12,4),expand=c(0,0)) +
#     scale_x_continuous(limits=c(0,100),breaks=seq(0,100,25)) +
    scale_fill_manual(values=c('#a8e0df','#67c9c7','#27b2af')) +
    theme_classic() +
    theme(axis.title.y = element_blank(),
          axis.text.y = element_blank(),
          axis.ticks.y = element_blank(),
          axis.title.x = element_blank(),
          axis.text.x = element_text(size=12,color='black'),
          axis.ticks.x = element_blank(),
          axis.line.y=element_blank(),
          legend.text.align=0,
         legend.key.size=unit(1,'cm'),
         legend.title = element_text(size=16),
         legend.text = element_text(size=16),
         legend.position='none')
    
    if(yax==FALSE){
        g1 <- g1+ylab('')
    }
    
    g4 <- g1+g2+g3
    
    return(g4)
}

In [ ]:
sliderPlotWrapper_2 <- function(dat,model,yax=TRUE){
    dat$predLog <- predict(model,dat,se.fit=TRUE)$fit
    dat$seLog <- predict(model,dat,se.fit=TRUE)$se.fit
    dat[,predAbs:=unlogit(predLog)]
    dat[,predUpper:=unlogit(predLog+(1.96*seLog))]
    dat[,predLower:=unlogit(predLog-(1.96*seLog))]
    
    g1 <- ggplot(dat[indepVar=='VATadjBMI'],aes(x=indepVar,y=predAbs*100,fill=Percentile)) +
    geom_bar(stat='identity',position='dodge') +
    geom_errorbar(aes(ymin=predLower*100,ymax=predUpper*100),
                  width=0.25,position=position_dodge(width=0.9),color='black',alpha=0.2)+
    geom_text(aes(label=format(round(predAbs*100,1),nsmall=1)),position=position_dodge(width=0.9),vjust=-0.8,size=annotatesize)+
    theme_classic() +
    xlab('') +
    ylab('Coronary Artery Disease, %') +
    scale_y_continuous(limits=c(0,18),breaks=seq(0,16,4),expand=c(0,0)) +
#     scale_x_continuous(limits=c(0,100),breaks=seq(0,100,25)) +
    scale_fill_manual(values=c('#f4adc8','#ec709f','#e53377')) +
    theme_classic() +
    theme(axis.title.y = element_text(size=18),
          axis.text.y = element_text(size=14,color='black'),
          axis.ticks.y = element_blank(),
          axis.title.x = element_text(size=18),
          axis.text.x = element_text(size=12,color='black'),
          axis.ticks.x = element_blank(),
          legend.text.align=0,
         legend.key.size=unit(1,'cm'),
         legend.title = element_text(size=16),
         legend.text = element_text(size=16),
         legend.position='none')
    
    g2 <- ggplot(dat[indepVar=='ASATadjBMI'],aes(x=indepVar,y=predAbs*100,fill=Percentile)) +
    geom_bar(stat='identity',position='dodge') +
    geom_errorbar(aes(ymin=predLower*100,ymax=predUpper*100),
                  width=0.25,position=position_dodge(width=0.9),color='black',alpha=0.2)+
    geom_text(aes(label=format(round(predAbs*100,1),nsmall=1)),position=position_dodge(width=0.9),vjust=-0.8,size=annotatesize)+
    theme_classic() +
    xlab('') +
    ylab('') +
    scale_y_continuous(limits=c(0,18),breaks=seq(0,16,4),expand=c(0,0)) +
#     scale_x_continuous(limits=c(0,100),breaks=seq(0,100,25)) +
    scale_fill_manual(values=c('#f7e0a5','#f1c962','#ecb31f')) +
    theme_classic() +
    theme(axis.title.y = element_blank(),
          axis.text.y = element_blank(),
          axis.ticks.y = element_blank(),
          axis.title.x = element_blank(),
          axis.text.x = element_text(size=12,color='black'),
          axis.ticks.x = element_blank(),
          axis.line.y=element_blank(),
          legend.text.align=0,
         legend.key.size=unit(1,'cm'),
         legend.title = element_text(size=16),
         legend.text = element_text(size=16),
         legend.position='none')
    
    g3 <- ggplot(dat[indepVar=='GFATadjBMI'],aes(x=indepVar,y=predAbs*100,fill=Percentile)) +
    geom_bar(stat='identity',position='dodge') +
    geom_errorbar(aes(ymin=predLower*100,ymax=predUpper*100),
                  width=0.25,position=position_dodge(width=0.9),color='black',alpha=0.2)+
    geom_text(aes(label=format(round(predAbs*100,1),nsmall=1)),position=position_dodge(width=0.9),vjust=-0.8,size=annotatesize)+
    theme_classic() +
    xlab('') +
    ylab('') +
    scale_y_continuous(limits=c(0,18),breaks=seq(0,16,4),expand=c(0,0)) +
#     scale_x_continuous(limits=c(0,100),breaks=seq(0,100,25)) +
    scale_fill_manual(values=c('#a8e0df','#67c9c7','#27b2af')) +
    theme_classic() +
    theme(axis.title.y = element_blank(),
          axis.text.y = element_blank(),
          axis.ticks.y = element_blank(),
          axis.title.x = element_blank(),
          axis.text.x = element_text(size=12,color='black'),
          axis.ticks.x = element_blank(),
          axis.line.y=element_blank(),
          legend.text.align=0,
         legend.key.size=unit(1,'cm'),
         legend.title = element_text(size=16),
         legend.text = element_text(size=16),
         legend.position='none')
    
    if(yax==FALSE){
        g1 <- g1+ylab('')
    }
    
    g4 <- g1+g2+g3
    
    return(g4)
}

In [ ]:
sliderPlotWrapper_2_female <- function(dat,model,yax=TRUE){
    dat$predLog <- predict(model,dat,se.fit=TRUE)$fit
    dat$seLog <- predict(model,dat,se.fit=TRUE)$se.fit
    dat[,predAbs:=unlogit(predLog)]
    dat[,predUpper:=unlogit(predLog+(1.96*seLog))]
    dat[,predLower:=unlogit(predLog-(1.96*seLog))]
    
    g1 <- ggplot(dat[indepVar=='VATadjBMI'],aes(x=indepVar,y=predAbs*100,fill=Percentile)) +
    geom_bar(stat='identity',position='dodge') +
    geom_errorbar(aes(ymin=predLower*100,ymax=predUpper*100),
                  width=0.25,position=position_dodge(width=0.9),color='black',alpha=0.2)+
    geom_text(aes(label=format(round(predAbs*100,1),nsmall=1)),position=position_dodge(width=0.9),vjust=-0.8,size=annotatesize)+
    theme_classic() +
    xlab('') +
    ylab('Coronary Artery Disease, %') +
    scale_y_continuous(limits=c(0,8*1.125),breaks=seq(0,8,4),expand=c(0,0)) +
#     scale_x_continuous(limits=c(0,100),breaks=seq(0,100,25)) +
    scale_fill_manual(values=c('#f4adc8','#ec709f','#e53377')) +
    theme_classic() +
    theme(axis.title.y = element_text(size=18),
          axis.text.y = element_text(size=14,color='black'),
          axis.ticks.y = element_blank(),
          axis.title.x = element_text(size=18),
          axis.text.x = element_text(size=12,color='black'),
          axis.ticks.x = element_blank(),
          legend.text.align=0,
         legend.key.size=unit(1,'cm'),
         legend.title = element_text(size=16),
         legend.text = element_text(size=16),
         legend.position='none')
    
    g2 <- ggplot(dat[indepVar=='ASATadjBMI'],aes(x=indepVar,y=predAbs*100,fill=Percentile)) +
    geom_bar(stat='identity',position='dodge') +
    geom_errorbar(aes(ymin=predLower*100,ymax=predUpper*100),
                  width=0.25,position=position_dodge(width=0.9),color='black',alpha=0.2)+
    geom_text(aes(label=format(round(predAbs*100,1),nsmall=1)),position=position_dodge(width=0.9),vjust=-0.8,size=annotatesize)+
    theme_classic() +
    xlab('') +
    ylab('') +
    scale_y_continuous(limits=c(0,8*1.125),breaks=seq(0,8,4),expand=c(0,0)) +
#     scale_x_continuous(limits=c(0,100),breaks=seq(0,100,25)) +
    scale_fill_manual(values=c('#f7e0a5','#f1c962','#ecb31f')) +
    theme_classic() +
    theme(axis.title.y = element_blank(),
          axis.text.y = element_blank(),
          axis.ticks.y = element_blank(),
          axis.title.x = element_blank(),
          axis.text.x = element_text(size=12,color='black'),
          axis.ticks.x = element_blank(),
          axis.line.y=element_blank(),
          legend.text.align=0,
         legend.key.size=unit(1,'cm'),
         legend.title = element_text(size=16),
         legend.text = element_text(size=16),
         legend.position='none')
    
    g3 <- ggplot(dat[indepVar=='GFATadjBMI'],aes(x=indepVar,y=predAbs*100,fill=Percentile)) +
    geom_bar(stat='identity',position='dodge') +
    geom_errorbar(aes(ymin=predLower*100,ymax=predUpper*100),
                  width=0.25,position=position_dodge(width=0.9),color='black',alpha=0.2)+
    geom_text(aes(label=format(round(predAbs*100,1),nsmall=1)),position=position_dodge(width=0.9),vjust=-0.8,size=annotatesize)+
    theme_classic() +
    xlab('') +
    ylab('') +
    scale_y_continuous(limits=c(0,8*1.125),breaks=seq(0,8,4),expand=c(0,0)) +
#     scale_x_continuous(limits=c(0,100),breaks=seq(0,100,25)) +
    scale_fill_manual(values=c('#a8e0df','#67c9c7','#27b2af')) +
    theme_classic() +
    theme(axis.title.y = element_blank(),
          axis.text.y = element_blank(),
          axis.ticks.y = element_blank(),
          axis.title.x = element_blank(),
          axis.text.x = element_text(size=12,color='black'),
          axis.ticks.x = element_blank(),
          axis.line.y=element_blank(),
          legend.text.align=0,
         legend.key.size=unit(1,'cm'),
         legend.title = element_text(size=16),
         legend.text = element_text(size=16),
         legend.position='none')
    
    if(yax==FALSE){
        g1 <- g1+ylab('')
    }
    
    g4 <- g1+g2+g3
    
    return(g4)
}

In [ ]:
maleDat <- rbind(sliderDatWrapper('Normal','Male',trainDat),
                sliderDatWrapper('Overweight','Male',trainDat),
                sliderDatWrapper('Obese','Male',trainDat))

femaleDat <- rbind(sliderDatWrapper('Normal','Female',trainDat),
                sliderDatWrapper('Overweight','Female',trainDat),
                sliderDatWrapper('Obese','Female',trainDat))

In [ ]:
maleDat$predT2D <- predict(prev_T2D_plotter_Male,maleDat,se.fit=TRUE)$fit
maleDat$seT2D <- predict(prev_T2D_plotter_Male,maleDat,se.fit=TRUE)$se.fit
maleDat[,predT2D_abs:=unlogit(predT2D)]
maleDat[,predT2D_Lower:=unlogit(predT2D-(1.96*seT2D))]
maleDat[,predT2D_Upper:=unlogit(predT2D+(1.96*seT2D))]

maleDat$predCAD <- predict(prev_CAD_plotter_Male,maleDat,se.fit=TRUE)$fit
maleDat$seCAD <- predict(prev_CAD_plotter_Male,maleDat,se.fit=TRUE)$se.fit
maleDat[,predCAD_abs:=unlogit(predCAD)]
maleDat[,predCAD_Lower:=unlogit(predCAD-(1.96*seCAD))]
maleDat[,predCAD_Upper:=unlogit(predCAD+(1.96*seCAD))]

In [ ]:
femaleDat$predT2D <- predict(prev_T2D_plotter_Female,femaleDat,se.fit=TRUE)$fit
femaleDat$seT2D <- predict(prev_T2D_plotter_Female,femaleDat,se.fit=TRUE)$se.fit
femaleDat[,predT2D_abs:=unlogit(predT2D)]
femaleDat[,predT2D_Lower:=unlogit(predT2D-(1.96*seT2D))]
femaleDat[,predT2D_Upper:=unlogit(predT2D+(1.96*seT2D))]

femaleDat$predCAD <- predict(prev_CAD_plotter_Female,femaleDat,se.fit=TRUE)$fit
femaleDat$seCAD <- predict(prev_CAD_plotter_Female,femaleDat,se.fit=TRUE)$se.fit
femaleDat[,predCAD_abs:=unlogit(predCAD)]
femaleDat[,predCAD_Lower:=unlogit(predCAD-(1.96*seCAD))]
femaleDat[,predCAD_Upper:=unlogit(predCAD+(1.96*seCAD))]

In [ ]:
for(i in c('Male')){
    yaxIn <- TRUE
    for(j in c('Normal','Overweight','Obese')){
        if(j != 'Normal'){
            yaxIn <- FALSE
        }
        assign(paste0('plot','_',i,'_',j),
               sliderPlotWrapper_1(sliderDatWrapper(j,i,trainDat),
                                   eval(as.name(paste0('prev_T2D_plotter_',i))),yax=yaxIn))
    }
}

for(i in c('Female')){
    yaxIn <- TRUE
    for(j in c('Normal','Overweight','Obese')){
        if(j != 'Normal'){
            yaxIn <- FALSE
        }
        assign(paste0('plot','_',i,'_',j),
               sliderPlotWrapper_1_female(sliderDatWrapper(j,i,trainDat),
                                   eval(as.name(paste0('prev_T2D_plotter_',i))),yax=yaxIn))
    }
}

T2D_plots <- ((plot_Male_Normal)|(plot_Male_Overweight)|(plot_Male_Obese))/(plot_Female_Normal|(plot_Female_Overweight)|(plot_Female_Obese))
options(repr.plot.width=16,repr.plot.height=8)
T2D_plots

# ggsave('/medpop/esp2/sagrawal/fat_distribution/2021.03.11_paperFigs/paper1_MLandEpi/revisions/quintilePlot_T2D_both_sex.pdf',
#       T2D_plots,height=8,width=16)

In [ ]:
for(i in c('Male')){
    yaxIn <- TRUE
    for(j in c('Normal','Overweight','Obese')){
        if(j != 'Normal'){
            yaxIn <- FALSE
        }
        assign(paste0('plot','_',i,'_',j),
               sliderPlotWrapper_2(sliderDatWrapper(j,i,trainDat),
                                   eval(as.name(paste0('prev_CAD_plotter_',i))),yax=yaxIn))
    }
}

for(i in c('Female')){
    yaxIn <- TRUE
    for(j in c('Normal','Overweight','Obese')){
        if(j != 'Normal'){
            yaxIn <- FALSE
        }
        assign(paste0('plot','_',i,'_',j),
               sliderPlotWrapper_2_female(sliderDatWrapper(j,i,trainDat),
                                   eval(as.name(paste0('prev_CAD_plotter_',i))),yax=yaxIn))
    }
}

CAD_plots <- ((plot_Male_Normal)|(plot_Male_Overweight)|(plot_Male_Obese))/(plot_Female_Normal|plot_Female_Overweight|plot_Female_Obese)
options(repr.plot.width=16,repr.plot.height=8)
CAD_plots

# ggsave('/medpop/esp2/sagrawal/fat_distribution/2021.03.11_paperFigs/paper1_MLandEpi/revisions/quintilePlot_CAD_both_sex.pdf',
#       CAD_plots,height=8,width=16)

## Incident T2D/CAD analysis

In [ ]:
fatPhenoCovar_202109 <- fread('/medpop/esp2/sagrawal/fat_distribution/2021.03.11_datafreeze/fatPhenoCovar_202109outcome.csv')



In [ ]:
fatPhenoCovar_202109[sex=='Male',vatadjbmi:=residuals(lm(vat~bmiCalc,fatPhenoCovar_202109[sex=='Male'],na.action = na.exclude))]
fatPhenoCovar_202109[sex=='Female',vatadjbmi:=residuals(lm(vat~bmiCalc,fatPhenoCovar_202109[sex=='Female'],na.action = na.exclude))]

fatPhenoCovar_202109[sex=='Male',asatadjbmi:=residuals(lm(asat~bmiCalc,fatPhenoCovar_202109[sex=='Male'],na.action = na.exclude))]
fatPhenoCovar_202109[sex=='Female',asatadjbmi:=residuals(lm(asat~bmiCalc,fatPhenoCovar_202109[sex=='Female'],na.action = na.exclude))]

fatPhenoCovar_202109[sex=='Male',gfatadjbmi:=residuals(lm(gfat~bmiCalc,fatPhenoCovar_202109[sex=='Male'],na.action = na.exclude))]
fatPhenoCovar_202109[sex=='Female',gfatadjbmi:=residuals(lm(gfat~bmiCalc,fatPhenoCovar_202109[sex=='Female'],na.action = na.exclude))]

fatNames <- c('vat','asat','gfat','vatadjbmi','asatadjbmi','gfatadjbmi','bmiCalc')

for(i in 1:length(fatNames)){
    fatPhenoCovar_202109[,paste0(fatNames[i],'Scaled'):=scale(eval(as.name(fatNames[i]))),by=sex]
}

fatPhenoCovar_202109[,mriNum_141243:=ifelse(mriNum==141243,1,0)]
fatPhenoCovar_202109[,mriNum_141631:=ifelse(mriNum==141631,1,0)]

In [ ]:
inc_T2D_surv <- coxph(Surv(fu_T2D_202109,inc_T2D_202109)~age_instance2+
                     sex+
                     bmiCalcScaled+
                     vatadjbmiScaled+asatadjbmiScaled+gfatadjbmiScaled+
                      mriNum_141243+mriNum_141631,
                     data=fatPhenoCovar_202109)

summary(inc_T2D_surv)

inc_CAD_surv <- coxph(Surv(fu_CAD_int_202109,inc_CAD_int_202109)~age_instance2+
                     sex+
                     bmiCalcScaled+
                     vatadjbmiScaled+asatadjbmiScaled+gfatadjbmiScaled+
                      mriNum_141243+mriNum_141631,
                     data=fatPhenoCovar_202109)

summary(inc_CAD_surv)

## Exercise and diet analyses

In [ ]:
modelbypredictormatrix <- function(models,predictors,predictoralias){
    #Expect 3 character vectors
    outdat <- data.table()
    for(i in 1:length(models)){
        for(j in 1:length(predictors)){
            
        tempCoef <- summary(eval(as.name(models[i])))$coefficients
        tempPredictor <- predictors[j]
        tempPredictorAlias <- predictoralias[j]
            

        indexVal <- which(row.names(tempCoef)==paste0(tempPredictor))
#         print(indexVal)
            
        tempBeta <- tempCoef[indexVal,1]
#         print(tempBeta)
        tempSE <- tempCoef[indexVal,2]
        tempUpper <- tempCoef[indexVal,1]+(1.96*tempSE)
        tempLower <- tempCoef[indexVal,1]-(1.96*tempSE)
        tempBetawCI <- paste0(format(round(tempBeta,2),nsmall=2),
                            ' ',
                            '(',
                            format(round(tempLower,2),nsmall=2),
                            ', ',
                            format(round(tempUpper,2),nsmall=2),
                            ')')
        tempPval <- tempCoef[indexVal,4]
#         print(tempPval)
        if(tempPval >= 0.01){
            tempPvalPrint <- round(tempPval,2)
        } else if(tempPval==0){
            tempPvalPrint <- '<1.0e-300'
        } else {
            tempPvalPrint <- formatC(tempPval,format='e',digits=1)
        }
            
        tempdat <- data.table('model'=models[i],
                              'predictor'=tempPredictorAlias,
                              'beta'=tempBeta,
                              'SE'=tempSE,
                              'upper'=tempUpper,
                              'lower'=tempLower,
                             'BetawCI'=tempBetawCI,
                              'Pval'=tempPval,
                              'PvalPrint'=tempPvalPrint)
            
        outdat <- rbind(outdat,tempdat)
        }
    }
    
    return(outdat)
}

In [ ]:
#Same quantities as above, just remade during revision with stratified diet and activity

fatPhenoCovarLifestyle3 <- fread('/medpop/esp2/sagrawal/fat_distribution/2021.03.11_datafreeze/fatPhenoCovarWithLifestyle2_onescript.csv')
saaketSumm(fatPhenoCovarLifestyle3)


In [ ]:
#Competing same quantities as above

fatPhenoCovarLifestyle3[sex=='Male',vatadjbmi:=residuals(lm(vat~bmiCalc,fatPhenoCovarLifestyle3[sex=='Male'],na.action = na.exclude))]
fatPhenoCovarLifestyle3[sex=='Female',vatadjbmi:=residuals(lm(vat~bmiCalc,fatPhenoCovarLifestyle3[sex=='Female'],na.action = na.exclude))]

fatPhenoCovarLifestyle3[sex=='Male',asatadjbmi:=residuals(lm(asat~bmiCalc,fatPhenoCovarLifestyle3[sex=='Male'],na.action = na.exclude))]
fatPhenoCovarLifestyle3[sex=='Female',asatadjbmi:=residuals(lm(asat~bmiCalc,fatPhenoCovarLifestyle3[sex=='Female'],na.action = na.exclude))]

fatPhenoCovarLifestyle3[sex=='Male',gfatadjbmi:=residuals(lm(gfat~bmiCalc,fatPhenoCovarLifestyle3[sex=='Male'],na.action = na.exclude))]
fatPhenoCovarLifestyle3[sex=='Female',gfatadjbmi:=residuals(lm(gfat~bmiCalc,fatPhenoCovarLifestyle3[sex=='Female'],na.action = na.exclude))]

fatNames <- names(fatPhenoCovarLifestyle3)[c(2:14)]

for(i in 1:length(fatNames)){
    fatPhenoCovarLifestyle3[,paste0(fatNames[i],'Scaled'):=scale(eval(as.name(fatNames[i]))),by=sex]
}

fatPhenoCovarLifestyle3[,mriNum_141243:=ifelse(mriNum==141243,1,0)]
fatPhenoCovarLifestyle3[,mriNum_141631:=ifelse(mriNum==141631,1,0)]

In [ ]:
table(fatPhenoCovarLifestyle3$dietStrat,useNA='always')
table(fatPhenoCovarLifestyle3$activityStrat,useNA='always')
table(fatPhenoCovarLifestyle3$smokingStrat,useNA='always')

In [ ]:
vatadjbmi_lifestyle <- lm(vatadjbmiScaled~age_instance2+
                      sex+mriNum_141243+mriNum_141631+dietStrat+factor(activityStrat)+smokingStrat,
                      data=fatPhenoCovarLifestyle3)

# summary(vatadjbmi_lifestyle)

asatadjbmi_lifestyle <- lm(asatadjbmiScaled~age_instance2+
                      sex+mriNum_141243+mriNum_141631+dietStrat+factor(activityStrat)+smokingStrat,
                      data=fatPhenoCovarLifestyle3)

# summary(asatadjbmi_lifestyle)

gfatadjbmi_lifestyle <- lm(gfatadjbmiScaled~age_instance2+
                      sex+mriNum_141243+mriNum_141631+dietStrat+factor(activityStrat)+smokingStrat,
                      data=fatPhenoCovarLifestyle3)

# summary(gfatadjbmi_lifestyle)

vat_lifestyle <- lm(vatScaled~age_instance2+
                      sex+mriNum_141243+mriNum_141631+dietStrat+factor(activityStrat)+smokingStrat,
                      data=fatPhenoCovarLifestyle3)

# summary(vat_lifestyle)

asat_lifestyle <- lm(asatScaled~age_instance2+
                      sex+mriNum_141243+mriNum_141631+dietStrat+factor(activityStrat)+smokingStrat,
                      data=fatPhenoCovarLifestyle3)

# summary(asat_lifestyle)

gfat_lifestyle <- lm(gfatScaled~age_instance2+
                      sex+mriNum_141243+mriNum_141631+dietStrat+factor(activityStrat)+smokingStrat,
                      data=fatPhenoCovarLifestyle3)

# summary(gfat_lifestyle)

vat_lifestyle_liters <- lm(vat~age_instance2+
                      sex+mriNum_141243+mriNum_141631+dietStrat+factor(activityStrat)+smokingStrat,
                      data=fatPhenoCovarLifestyle3)

# summary(vat_lifestyle_liters)

asat_lifestyle_liters <- lm(asat~age_instance2+
                      sex+mriNum_141243+mriNum_141631+dietStrat+factor(activityStrat)+smokingStrat,
                      data=fatPhenoCovarLifestyle3)

# summary(asat_lifestyle_liters)

gfat_lifestyle_liters <- lm(gfat~age_instance2+
                      sex+mriNum_141243+mriNum_141631+dietStrat+factor(activityStrat)+smokingStrat,
                      data=fatPhenoCovarLifestyle3)

# summary(gfat_lifestyle_liters)

In [ ]:
printdat <- modelbypredictormatrix(models=c('vatadjbmi_lifestyle',
                               'asatadjbmi_lifestyle',
                               'gfatadjbmi_lifestyle',
                               'vat_lifestyle',
                               'asat_lifestyle',
                               'gfat_lifestyle',
                               'vat_lifestyle_liters',
                               'asat_lifestyle_liters',
                               'gfat_lifestyle_liters'),
                      predictors=c('dietStrat',
                                  'factor(activityStrat)1',
                                  'factor(activityStrat)2'),
                      predictoralias=c('Ideal Diet',
                                      'Intermediate Exercise',
                                      'Ideal Exercise'))

printdat <- printdat[,c('model','predictor','BetawCI','PvalPrint')]
printdat

# write.csv(printdat,
#          './2021.03.11_paperFigs/paper1_MLandEpi/revisions/lifestyle_betas.csv',
#          row.names=FALSE)